## install prado needed software 
can take 15 minutes, it's worth choosing gpu runtime in google colab

In [ ]:
!sudo apt install curl gnupg
!curl https://bazel.build/bazel-release.pub.gpg | sudo apt-key add -
!echo "deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo tee /etc/apt/sources.list.d/bazel.list
!sudo apt update
!sudo apt install bazel=4.2.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.16).
gnupg is already the newest version (2.2.4-1ubuntu1.4).
0 upgraded, 0 newly installed, 0 to remove and 76 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3199  100  3199    0     0  35544      0 --:--:-- --:--:-- --:--:-- 35544
OK
deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8
Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://storage.googleapis.com/bazel-apt stable InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bi

In [ ]:
!git clone https://www.github.com/tensorflow/models
!models/research/seq_flow_lite/demo/colab/setup_workspace.sh
!pip install models/research/seq_flow_lite
!rm -rf models/research/seq_flow_lite/tf_ops
!rm -rf models/research/seq_flow_lite/tflite_ops

Cloning into 'models'...
remote: Enumerating objects: 68753, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 68753 (delta 16), reused 27 (delta 14), pack-reused 68721
Receiving objects: 100% (68753/68753), 577.08 MiB | 32.58 MiB/s, done.
Resolving deltas: 100% (48449/48449), done.
Processing ./models/research/seq_flow_lite
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for seq-flow-lite: filename=seq_flow_lite-0.1-py3-none-any.whl size=772074 sha256=10d3cb223c80ac9552775304dd3801fc4d968401212f5e2eeb303ce6109f477d
  Stored in directory: /tmp/pip-ephem-wheel-cache

In [ ]:
# %cd content/
%cd models/research/seq_flow_lite
from layers import base_layers
from layers import projection_layers
from models import prado


/content/models/research/seq_flow_lite


In [ ]:
!pip install gdown

!gdown --id 1kbYmGkMsQGVBdnDStxx3nz79WNh5JXy7 # training_set_clean_only_text.txt
!gdown --id 1XP30I7gKxhY1jOCOFlcIWDQDnZ3Qk9rZ # training_set_clean_only_tags.txt
!gdown --id 1z0Laz8jCGR-GQcMNL2_qKsGnbL4VRRCY # test_set_clean_only_text.txt
!gdown --id 1o_XRCnKlScCrZRk5suzjwzljT7YonQ2q # test_set_clean_only_tags.txt


!gdown --id 1gJL8GYTuWj5BXUUFnCy--AE4xEZlDr1u # corpora

!gdown --id 1-IeRYiTy5rujiTg97jrEzhCSpdJkoBL1 # dataset4

!gdown --id 1IezKpxY0c8OHv0nq44yrUH8pYtHnsnY- # dataset5

## imports

In [ ]:
!pip install morfeusz2
import morfeusz2
morf = morfeusz2.Morfeusz()
import pandas as pd
import numpy as np
import re

import string
from string import ascii_lowercase
from sklearn.metrics import f1_score, confusion_matrix

import nltk
nltk.download('punkt')
import unicodedata
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import gensim
from sklearn.model_selection import train_test_split

!pip install emoji
import emoji
from string import punctuation

import pickle

!pip install glove-python-binary
import glove

import gdown


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## read poleval datasets

In [ ]:
def read_file(file_name): # read given file and create list of phrases
  lines = []
  with open(file_name, encoding="UTF-8-SIG") as text:
    for line in text:
      lines.append(line[:-1])
  return lines

  # merge texts with labels
def merge_lists(list1, list2):
  merged = []
  for i in range(len(list1)):
    merged.append([list1[i], list2[i]])
  return merged
  
text = read_file('training_set_clean_only_text.txt')
labels = read_file('training_set_clean_only_tags.txt')

texts_labels = merge_lists(text, labels)

text_test = read_file('test_set_clean_only_text.txt')
labels_test = read_file('test_set_clean_only_tags.txt')

texts_labels_test = merge_lists(text_test, labels_test)


Y_train = [line[1] for line in texts_labels if line[0][:2] != 'RT']
Y_test = [line[1] for line in texts_labels_test if line[0][:2] != 'RT'] # remove retweets

## utility functions

### functions prado

In [ ]:
def create_dict_for_tf(texts, labels):
  labels_dict = ['text', 'hate']
  list_dict_to_tensor = []
  for text, lab in zip(texts, labels):
    value_hate = False
    if lab == '1':
      value_hate = True
    list_dict_to_tensor.append({'text':text, 'hate':value_hate})
  # return list_dict_to_tensor
  return tf.data.Dataset.from_tensor_slices(pd.DataFrame.from_dict(list_dict_to_tensor).to_dict(orient="list"))


In [ ]:
def build_dataset_poleval(mode, train_set, test_set, inspect=False):
  if mode == base_layers.TRAIN:
    data_file = train_set
    count = None
  elif mode == base_layers.EVAL:
    data_file = test_set
    count = 1
  else:
    raise ValueError('mode={}, must be TRAIN or EVAL'.format(mode))

  batch_size = CONFIG['batch_size']
  if inspect:
    batch_size = 1

  def process_input(features):
    text = features['text']
    text = tf.reshape(text, [batch_size])
    projection_layer = projection_layers.ProjectionLayer(MODEL_CONFIG, mode)
    projection, seq_length = projection_layer(text)

    label = tf.stack([features[label] for label in LABELS], axis=-1)
    label = tf.cast(label, tf.float32)
    label = tf.reshape(label, [batch_size, len(LABELS)])
    model_features = ({'projection': projection, 'sequence_length': seq_length}, label)

    if inspect:
      model_features = (model_features[0], model_features[1], features)

    return model_features

  ds = data_file

  if mode == base_layers.EVAL:
    ds = ds.batch(batch_size, drop_remainder=True).repeat(count=count)

  if mode == base_layers.TRAIN:
    ds = ds.repeat(count=count)
    ds = ds.shuffle(buffer_size=batch_size * 2)
    ds = ds.batch(batch_size, drop_remainder=True)
  if mode == base_layers.EVAL:
    ds = ds.map(process_input,
              num_parallel_calls=tf.data.experimental.AUTOTUNE,
              deterministic=True)
  else:
    ds = ds.map(process_input,
              num_parallel_calls=tf.data.experimental.AUTOTUNE,
              deterministic=False)
  
  ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  return ds


### basic processing

In [ ]:
not_wanted_words = ['@anonymized_account']

def word_process(word, remove_accents=False, keep_punctuation=False):
  if word in not_wanted_words:
    return False

  emoji_pattern = re.compile("["
          u"\U0001F600-\U0001F64F"  # emoticons
          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
          u"\U0001F680-\U0001F6FF"  # transport & map symbols
          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+", flags=re.UNICODE)
  word = emoji_pattern.sub(r'', word) # no emoji
  word = word.lower()
  if not keep_punctuation:
    word = word.translate(str.maketrans('', '', string.punctuation))

  analysis = morf.analyse(word)
  for interpretation in analysis:
    word = interpretation[2][1].split(':')[0]
    break 
  word = word.lower()
  if len(word)==0:
    return False
  if remove_accents == True:
    word = unicodedata.normalize('NFKD', word)
  return word


def clean_dataset(dataset, remove_accents=False, keep_punctuation=False):
  dataset_clean = []
  for line in dataset:

    row = []
    if keep_punctuation:
      words = nltk.word_tokenize(line[0])
    else:
      words = line[0].split()
    if words[0] == 'RT': 
      continue
    for word in words:

      word = word_process(word, remove_accents, keep_punctuation)
      if word != False: 
        row.append(word)
    dataset_clean.append(' '.join(row))
  return dataset_clean

def create_dict_for_tf(texts, labels):
  labels_dict = ['text', 'hate']
  list_dict_to_tensor = []
  for text, lab in zip(texts, labels):
    value_hate = False
    if lab == '1':
      value_hate = True
    list_dict_to_tensor.append({'text':text, 'hate':value_hate})
  return tf.data.Dataset.from_tensor_slices(pd.DataFrame.from_dict(list_dict_to_tensor).to_dict(orient="list"))


check max sentences length

In [ ]:
max_len = 0
for sen in clean_train:
  new_sen = sen.split()
  if len(new_sen) > max_len:
    max_len = len(new_sen)
print(max_len)
max_len = 0
for sen in clean_test:
  new_sen = sen.split()
  if len(new_sen) > max_len:
    max_len = len(new_sen)
print(max_len)

29
27


### base functions for embeddings

In [ ]:
def build_dataset_poleval(mode, train_set, test_set, inspect=False):
  if mode == base_layers.TRAIN:
    data_file = train_set
    count = None
  elif mode == base_layers.EVAL:
    data_file = test_set
    count = 1
  else:
    raise ValueError('mode={}, must be TRAIN or EVAL'.format(mode))

  batch_size = CONFIG['batch_size']
  if inspect:
    batch_size = 1

  def process_input(features):
    text = features['text']
    text = tf.reshape(text, [batch_size])
    projection_layer = projection_layers.ProjectionLayer(MODEL_CONFIG, mode)
    projection, seq_length = projection_layer(text)

    label = tf.stack([features[label] for label in LABELS], axis=-1)
    label = tf.cast(label, tf.float32)
    label = tf.reshape(label, [batch_size, len(LABELS)])

    model_features = ({'projection': projection, 'sequence_length': seq_length}, label)

    if inspect:
      model_features = (model_features[0], model_features[1], features)

    return model_features

  ds = data_file

  if mode == base_layers.EVAL:
    ds = ds.batch(batch_size, drop_remainder=True).repeat(count=count)

  if mode == base_layers.TRAIN:
    ds = ds.repeat(count=count)
    ds = ds.shuffle(buffer_size=batch_size * 2)
    ds = ds.batch(batch_size, drop_remainder=True)
  if mode == base_layers.EVAL:
    ds = ds.map(process_input,
              num_parallel_calls=tf.data.experimental.AUTOTUNE,
              deterministic=True)
  else:
    ds = ds.map(process_input,
              num_parallel_calls=tf.data.experimental.AUTOTUNE,
              deterministic=False)
  
  ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  return ds


In [ ]:
def build_model(mode):
  inputs = []
  if mode == base_layers.TRAIN or mode == base_layers.EVAL:

    projection = tf.keras.Input(
        shape=(MODEL_CONFIG['max_seq_len'], MODEL_CONFIG['feature_size']),
        name='projection',
        dtype='float32')

    sequence_length = tf.keras.Input(
        shape=(), name='sequence_length', dtype='float32')
    inputs = [projection, sequence_length]
  else:
    print()

  print(sequence_length)
  model_layer = prado.Encoder(MODEL_CONFIG, mode)
  logits = model_layer(projection, sequence_length)

  if MODEL_CONFIG['multilabel']:
    activation = tf.keras.layers.Activation('softmax', name='predictions')
  else:
    activation = tf.keras.layers.Activation('sigmoid', name='predictions') 
  predictions = activation(logits)

  model = tf.keras.Model(
      inputs=inputs,
      outputs=[predictions])
  
  return model


In [ ]:
def create_metrics(labels):
  metrics = []
  for i, label in enumerate(labels):
    metric = tf.keras.metrics.Precision(thresholds=[0.5],class_id=i,name='precision@0.5/{}'.format(label))
    metrics.append(metric)
    metric = tf.keras.metrics.Recall(thresholds=[0.5],class_id=i,name='recall@0.5/{}'.format(label))
    metrics.append(metric)
    metric = tf.keras.metrics.TruePositives(thresholds=[0.5],name='TruePositives@0.5/{}'.format(label))
    metrics.append(metric)
    metric = tf.keras.metrics.TrueNegatives(thresholds=[0.5],name='TrueNegatives@0.5/{}'.format(label))
    metrics.append(metric)
    metric = tf.keras.metrics.FalsePositives(thresholds=[0.5],name='FalsePositives@0.5/{}'.format(label))
    metrics.append(metric)
    metric = tf.keras.metrics.FalseNegatives(thresholds=[0.5],name='FalseNegatives@0.5/{}'.format(label))
    metrics.append(metric)
  metric = tf.keras.metrics.Precision(thresholds=[0.5], name='precision@0.5/all')
  metrics.append(metric)
  metric = tf.keras.metrics.Recall(thresholds=[0.5], name='recall@0.5/all')
  metrics.append(metric)
  return metrics

def build_run_save_eval_model(train_data, test_data, config, labels, model_name):
  model = build_model(base_layers.TRAIN)

  learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
      initial_learning_rate=config['learning_rate'],
      decay_rate=config['learning_rate_decay_rate'],
      decay_steps=config['learning_rate_decay_steps'],
      staircase=True)
  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)

  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  epochs = int(config['train_steps'] / config['save_checkpoints_steps'])
  model.fit(
      x=train_data,
      epochs=epochs,
      steps_per_epoch=config['save_checkpoints_steps'])

  model.save_weights(f'{model_name}/model_checkpoint')
  model = build_model(base_layers.EVAL)
  metrics = create_metrics(labels)
  model.compile(metrics=metrics)
  model.load_weights(f'{model_name}/model_checkpoint')
  result = model.evaluate(x=test_data, return_dict=True)
  prec = result['precision@0.5/hate']
  rec = result['recall@0.5/all']
  f1 = 2 * (prec*rec) / (prec+rec)
  print(f1)


In [ ]:
def create_dict_for_tf_emb(texts, labels, embeddings, emb_len=100, max_num_words_sen=128, glove=False):
  labels_dict = ['text', 'hate']
  list_dict_to_tensor = []
  word_array = np.zeros([max_num_words_sen,emb_len])
  for text, lab in zip(texts, labels):
    value_hate = False
    if lab == '1':
      value_hate = True

    words = text.split()
    word_array = np.zeros([max_num_words_sen,emb_len])
    if glove:
      for i, word in enumerate(words):
        if word in embeddings.dictionary:
          word_array[i] = embeddings.word_vectors[embeddings.dictionary[word]]    
    else:
      for i , word in enumerate(words):
        if word in embeddings:
          word_array[i] = embeddings[word]  

    list_dict_to_tensor.append({'text':word_array, 'seq_len': len(words), 'hate':value_hate})
  return tf.data.Dataset.from_tensor_slices(pd.DataFrame.from_dict(list_dict_to_tensor).to_dict(orient="list"))

In [ ]:
def build_dataset_poleval_emb(mode, train_set, test_set, inspect=False):
  if mode == base_layers.TRAIN:
    data_file = train_set
    count = None
  elif mode == base_layers.EVAL:
    data_file = test_set
    count = 1
  else:
    raise ValueError('mode={}, must be TRAIN or EVAL'.format(mode))

  batch_size = CONFIG['batch_size']
  if inspect:
    batch_size = 1

  def process_input(features):

    text = features['text']
    print(len(text)) 
    print(f'type of features{type(features)} type of text {type(text)}')
    projection = features['text']
    seq_length = features['seq_len']
    print(projection.shape) # 128 128 512
    print(seq_length.shape) # 128

    label = tf.stack([features[label] for label in LABELS], axis=-1)
    label = tf.cast(label, tf.float32)
    label = tf.reshape(label, [batch_size, len(LABELS)])
    model_features = ({'projection': projection, 'sequence_length': seq_length}, label)
    if inspect:
      model_features = (model_features[0], model_features[1], features)
    return model_features

  ds = data_file

  if mode == base_layers.EVAL:
    ds = ds.batch(batch_size, drop_remainder=True).repeat(count=count)

  if mode == base_layers.TRAIN:
    ds = ds.repeat(count=count)
    ds = ds.shuffle(buffer_size=batch_size * 2)
    ds = ds.batch(batch_size, drop_remainder=True)
  # print(ds)
  if mode == base_layers.EVAL:
    ds = ds.map(process_input,
                num_parallel_calls=tf.data.experimental.AUTOTUNE,
                deterministic=True)  
  else:
    ds = ds.map(process_input,
              num_parallel_calls=tf.data.experimental.AUTOTUNE,
              deterministic=False)
  ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  return ds



In [ ]:
emojis_to_words = {'CONFOUNDED FACE': 'zmieszanie',
                  'CRYING FACE': 'płacz',
                  'DISAPPOINTED BUT RELIEVED FACE': 'rozczarowanie',
                  'DISAPPOINTED FACE': 'rozczarowanie',
                  'EXPRESSIONLESS FACE': 'bezwyrazowy',
                  'FACE SAVOURING DELICIOUS FOOD': 'pyszności',
                  'FACE SCREAMING IN FEAR': 'strach',
                  'FACE THROWING A KISS': 'pocałunek',
                  'FACE WITH COLD SWEAT': 'stres',
                  'FACE WITH MEDICAL MASK': 'maska',
                  'FACE WITH OK GESTURE': 'ok',
                  'FACE WITH OPEN MOUTH': 'zdziwienie',
                  'FACE WITH STUCK-OUT TONGUE': 'język',
                  'FACE WITH STUCK-OUT TONGUE AND TIGHTLY-CLOSED EYES': 'język',
                  'FACE WITH STUCK-OUT TONGUE AND WINKING EYE': 'język',
                  'FACE WITH TEARS OF JOY': 'radość',
                  'FEARFUL FACE': 'strach',
                  'FLUSHED FACE': 'zaczerwienienie',
                  'GRIMACING FACE': 'grymas',
                  'GRINNING FACE': 'uśmiech',
                  'GRINNING FACE WITH SMILING EYES': 'uśmiech',
                  'HAPPY PERSON RAISING ONE HAND': 'radość',
                  'HEAR-NO-EVIL MONKEY': 'małpka',
                  'HUSHED FACE': 'uciszenie',
                  'LOUDLY CRYING FACE': 'płacz',
                  'NEUTRAL FACE': 'neutralność',
                  'PENSIVE FACE': 'zamyślenie',
                  'PERSEVERING FACE': 'wytrwałość',
                  'PERSON BOWING DEEPLY': 'ukłon',
                  'PERSON RAISING BOTH HANDS IN CELEBRATION': 'radość',
                  'PERSON WITH FOLDED HANDS': 'ok',
                  'POUTING FACE': 'dąsanie',
                  'SEE-NO-EVIL MONKEY': 'małpka',
                  'SLIGHTLY SMILING FACE': 'uśmiech',
                  'SMILING FACE WITH HALO': 'uśmiech',
                  'SMILING FACE WITH HEART-SHAPED EYES': 'miłość',
                  'SMILING FACE WITH OPEN MOUTH': 'uśmiech',
                  'SMILING FACE WITH OPEN MOUTH AND COLD SWEAT': 'uśmiech',
                  'SMILING FACE WITH OPEN MOUTH AND SMILING EYES': 'uśmiech',
                  'SMILING FACE WITH OPEN MOUTH AND TIGHTLY-CLOSED EYES': 'uśmiech',
                  'SMILING FACE WITH SMILING EYES': 'uśmiech',
                  'SMILING FACE WITH SUNGLASSES': 'uśmiech',
                  'SPEAK-NO-EVIL MONKEY': 'małpka',
                  'UNAMUSED FACE': 'nierozbawienie',
                  'UPSIDE-DOWN FACE': 'uśmiech',
                  'WEARY FACE': 'zmęczenie',
                  'WINKING FACE': 'mrugnięcie'}

In [ ]:
not_wanted_words = ['@anonymized_account', 'account', 'anonymized' ]

emoji_pattern_no_emoji = re.compile("["
          u"\U0001F600-\U0001F64F"  # emoticons
          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
          u"\U0001F680-\U0001F6FF"  # transport & map symbols
          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
          u"\U0001F700-\U0001F77F"  # alchemical symbols
          u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
          u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
          u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
          u"\U0001FA00-\U0001FA6F"  # Chess Symbols
          u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
          u"\U00002702-\U000027B0"  # Dingbats
          u"\U000024C2-\U0001F251" 
                            "]+", flags=re.UNICODE)

emoji_pattern = re.compile("["
          # u"\U0001F600-\U0001F64F"  # emoticons
          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
          u"\U0001F680-\U0001F6FF"  # transport & map symbols
          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
          u"\U0001F700-\U0001F77F"  # alchemical symbols
          u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
          u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
          u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
          u"\U0001FA00-\U0001FA6F"  # Chess Symbols
          u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
          u"\U00002702-\U000027B0"  # Dingbats
          u"\U000024C2-\U0001F251" 
                            "]+", flags=re.UNICODE)

only_faces_emoji_pattern = re.compile("["
          u"\U0001F600-\U0001F64F"  # emoticons
                            "]+", flags=re.UNICODE)

def word_process_with_emoji(word, previous_word='', keep_emoji=False):
  if word in not_wanted_words:
    return False
  if keep_emoji == False:
    if emoji_pattern_no_emoji.match(word):
      return False
  else:
    if emoji_pattern.match(word):
      return False
  if only_faces_emoji_pattern.match(word):

    word = unicodedata.name(word)

    if word in emojis_to_words: 
      word = emojis_to_words[word]
    else: return False
    if word == previous_word:
      return False
    return word
  word = word.lower()
  analysis = morf.analyse(word)
  for interpretation in analysis:
    word = interpretation[2][1].split(':')[0]
    break 
  word = word.lower()
  if len(word)==0:
    return False
  return word


def clean_dataset_with_emoji(dataset, keep_emoji=False, print_raw_data=[]):
  dataset_clean = []
  re_split = re.compile(r'[\s{}]+'.format(re.escape(punctuation)))
  new_words = []
  for line in dataset:
    row = []
    
    words = re_split.split(line)
    if words[0] == 'RT': 
      continue
    new_words = []
    for word in words:
      if len(word) == 0: continue
      if not emoji_pattern.match(word) and not only_faces_emoji_pattern.match(word):
        new_words.append(word)
      elif len(word) == 1:
        new_words.append(word)
      else:
        word = emoji.get_emoji_regexp().split(word)
        word = [i for i in word if len(i)>0]
        new_words += word

    previous_word = ''
    for word in new_words:
      word = word_process_with_emoji(word, previous_word, keep_emoji)
      if word != False:
        previous_word = word
      if word != False: 
        row.append(word)
    if print_raw_data:
      if ' '.join(row) in print_raw_data:
        print(line)
    dataset_clean.append(' '.join(row))
  return dataset_clean



In [ ]:
def save_classification_results(model, test_dataset, clean_test, y_test, filename):
  pred = model.predict(x=test_dataset)
  pred  = [0 if i<0.5 else 1 for i in pred]

  Y_test_int = [int(i) for i in y_test]
  tp, tn, fp, fn = [],[],[],[]
  for i in range(len(pred)):
    if pred[i] == Y_test_int[i] and pred[i]==1:
      tp.append(clean_test[i])
    elif pred[i] == Y_test_int[i] and pred[i]==0:
      tn.append(clean_test[i])
    elif pred[i] == 0:
      fn.append(clean_test[i])
    else:
      fp.append(clean_test[i])
  dict_conf_matrix = {'tp':tp, 'tn':tn, 'fp':fp, 'fn':fn}
  with open(f'{filename}.pickle', 'wb') as handle:
    pickle.dump(dict_conf_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)


some code was based on provided sample notebooks in PRADO repository on github showing how to use the network

## original PRADO 

In [ ]:
LABELS = [
    'hate',
]
CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,
}
CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)
MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': False,
    'quantize': False,
    'max_seq_len': 128,
    'max_seq_len_inference': 128,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 512,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,
}
CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
clean_train = clean_dataset_with_emoji(text, keep_emoji=True)
clean_test = clean_dataset_with_emoji(text_test, keep_emoji=True)
train_hate = create_dict_for_tf(clean_train, Y_train)
test_hate = create_dict_for_tf(clean_test, Y_test)
train_dataset_hate = build_dataset_poleval(base_layers.TRAIN, train_hate, test_hate)
test_dataset_hate = build_dataset_poleval(base_layers.EVAL, train_hate, test_hate)

In [ ]:
# build_run_save_eval_model(train_dataset_hate, test_dataset_hate, CONFIG, LABELS, 'model_prado_base_pdf_no_emoji')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 25s 23ms/step - loss: 0.5961 - accuracy: 0.7426
Epoch 2/10
1000/1000 [==============================] - 24s 24ms/step - loss: 0.4646 - accuracy: 0.8790
Epoch 3/10
1000/1000 [==============================] - 23s 23ms/step - loss: 0.4306 - accuracy: 0.9067
Epoch 4/10
1000/1000 [==============================] - 24s 24ms/step - loss: 0.4186 - accuracy: 0.9177
Epoch 5/10
1000/1000 [==============================] - 24s 23ms/step - loss: 0.4150 - accuracy: 0.9167
Epoch 6/10
1000/1000 [==============================] - 24s 24ms/step - loss: 0.4130 - accuracy: 0.9195
Epoch 7/10
1000/1000 [==============================] - 24s 24ms/step - loss: 0.4122 - accuracy: 0.9204
Epoch 8/10
1000/1000 [==============================] - 24s 24ms/step - loss: 0.4121 - accuracy: 0.9199
Epo

In [3]:
gdown.download_folder('https://drive.google.com/drive/folders/1-c86tKaGw5bKs6L3_EZex0wMMLUPc6cj?usp=sharing', quiet=True)

['/content/model_prado_base_pdf_no_emoji/checkpoint',
 '/content/model_prado_base_pdf_no_emoji/model_checkpoint.data-00000-of-00001',
 '/content/model_prado_base_pdf_no_emoji/model_checkpoint.index']

In [ ]:
model_hate_eval = build_model(base_layers.EVAL)
metrics = create_metrics(LABELS)
model_hate_eval.compile(metrics=metrics)
model_hate_eval.load_weights('model_prado_base_pdf_no_emoji/model_checkpoint')
result = model_hate_eval.evaluate(x=test_dataset_hate, return_dict=True)

prec = result['precision@0.5/hate']
rec = result['recall@0.5/all']
f1 = 2 * (prec*rec) / (prec+rec)
print(f1)

max sen len 32, switch to data with emoji

In [ ]:
LABELS = [
    'hate',
]
CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,
}
CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)
MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': False,
    'quantize': False, 
    'max_seq_len': 32,
    'max_seq_len_inference': 32,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 512,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,
}
CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
clean_train = clean_dataset_with_emoji(text, keep_emoji=True)
clean_test = clean_dataset_with_emoji(text_test, keep_emoji=True)
train_hate = create_dict_for_tf(clean_train, Y_train)
test_hate = create_dict_for_tf(clean_test, Y_test)
train_dataset_hate = build_dataset_poleval(base_layers.TRAIN, train_hate, test_hate)
test_dataset_hate = build_dataset_poleval(base_layers.EVAL, train_hate, test_hate)

In [ ]:
# build_run_save_eval_model(train_dataset_hate, test_dataset_hate, CONFIG, LABELS, 'model_prado_base_pdf_emoji_seq32')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 18s 15ms/step - loss: 0.5955 - accuracy: 0.7376
Epoch 2/10
1000/1000 [==============================] - 13s 13ms/step - loss: 0.4657 - accuracy: 0.8787
Epoch 3/10
1000/1000 [==============================] - 13s 13ms/step - loss: 0.4310 - accuracy: 0.9085
Epoch 4/10
1000/1000 [==============================] - 15s 15ms/step - loss: 0.4194 - accuracy: 0.9155
Epoch 5/10
1000/1000 [==============================] - 20s 20ms/step - loss: 0.4159 - accuracy: 0.9170
Epoch 6/10
1000/1000 [==============================] - 18s 18ms/step - loss: 0.4136 - accuracy: 0.9211
Epoch 7/10
1000/1000 [==============================] - 13s 13ms/step - loss: 0.4128 - accuracy: 0.9203
Epoch 8/10
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4133 - accuracy: 0.9208
Epo

1024 features max sen len 128

In [ ]:
LABELS = [
    'hate',
]
CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,
}
CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)
MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': False,
    'quantize': False, 
    'max_seq_len': 128,
    'max_seq_len_inference': 128,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 1024,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,
}
CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
# build_run_save_eval_model(train_dataset_hate, test_dataset_hate, CONFIG, LABELS, 'model_prado_base_pdf_emoji_seq128_f1024')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10


1000/1000 [==============================] - 39s 35ms/step - loss: 0.5779 - accuracy: 0.7817
Epoch 2/10
1000/1000 [==============================] - 42s 42ms/step - loss: 0.4424 - accuracy: 0.9030
Epoch 3/10
1000/1000 [==============================] - 42s 42ms/step - loss: 0.4046 - accuracy: 0.9312
Epoch 4/10
1000/1000 [==============================] - 34s 34ms/step - loss: 0.3926 - accuracy: 0.9382
Epoch 5/10
1000/1000 [==============================] - 35s 35ms/step - loss: 0.3877 - accuracy: 0.9418
Epoch 6/10
1000/1000 [==============================] - 34s 34ms/step - loss: 0.3867 - accuracy: 0.9400
Epoch 7/10
1000/1000 [==============================] - 34s 34ms/step - loss: 0.3854 - accuracy: 0.9431
Epoch 8/10
1000/1000 [==============================] - 34s 34ms/step - loss: 0.3857 - accuracy: 0.9417
Epoch 9/10
1000/1000 [==============================] - 34s 34ms/step - loss: 0.3859 - accuracy: 0.9420
Epoch 10/10
1000/1000 [==============================] - 34s 34ms/step - lo

quantize 512 feat seq128

In [ ]:
LABELS = [
    'hate',
]
CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,
}
CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)
MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': False,
    'quantize': True,
    'max_seq_len': 128,
    'max_seq_len_inference': 128,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 512,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,
}
CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
# build_run_save_eval_model(train_dataset_hate, test_dataset_hate, CONFIG, LABELS, 'model_prado_base_pdf_emoji_quant')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 38s 34ms/step - loss: 0.5978 - accuracy: 0.7482
Epoch 2/10
1000/1000 [==============================] - 33s 33ms/step - loss: 0.4649 - accuracy: 0.8789
Epoch 3/10
1000/1000 [==============================] - 33s 33ms/step - loss: 0.4310 - accuracy: 0.9088
Epoch 4/10
1000/1000 [==============================] - 33s 33ms/step - loss: 0.4196 - accuracy: 0.9175
Epoch 5/10
1000/1000 [==============================] - 33s 33ms/step - loss: 0.4152 - accuracy: 0.9186
Epoch 6/10
1000/1000 [==============================] - 32s 32ms/step - loss: 0.4141 - accuracy: 0.9197
Epoch 7/10
1000/1000 [==============================] - 33s 33ms/step - loss: 0.4130 - accuracy: 0.9205
Epoch 8/10
1000/1000 [==============================] - 33s 33ms/step - loss: 0.4132 - accuracy: 0.9206
Epo

In [ ]:
(59+769)/945

0.8761904761904762

In [ ]:
LABELS = [
    'hate',
]
CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,
}
CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)
MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': False,
    'quantize': False,
    'max_seq_len': 128,
    'max_seq_len_inference': 128,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 512,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,
}
CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
# build_run_save_eval_model(train_dataset_hate, test_dataset_hate, CONFIG, LABELS, 'model_prado_base_pdf_emoji_quant')

### prado base + emojis

In [ ]:
model_hate_eval = build_model(base_layers.EVAL)
metrics = create_metrics(LABELS)
model_hate_eval.compile(metrics=metrics)
model_hate_eval.load_weights('model_prado_base_pdf_emoji/model_checkpoint')
result = model_hate_eval.evaluate(x=test_dataset_hate, return_dict=True)

prec = result['precision@0.5/hate']
rec = result['recall@0.5/all']
f1 = 2 * (prec*rec) / (prec+rec)
print(f1)

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
9/9 [==============================] - 2s 25ms/step - loss: 0.0000e+00 - precision@0.5/hate: 0.4926 - recall@0.5/hate: 0.5537 - TruePositives@0.5/hate: 67.0000 - TrueNegatives@0.5/hate: 755.0000 - FalsePositives@0.5/hate: 69.0000 - FalseNegatives@0.5/hate: 54.0000 - precision@0.5/all: 0.4926 - recall@0.5/all: 0.5537
0.521400763581715


In [ ]:
# save_classification_results(model_hate_eval, test_dataset_hate, clean_test, Y_test, 'prado_best')

In [ ]:
gdown.download_folder('https://drive.google.com/drive/folders/1-HJ-mpCVB8oPPyUvAGSbeessy_eWbfvw?usp=sharing', quiet=True)

['/content/model_prado_base_pdf_emoji/checkpoint',
 '/content/model_prado_base_pdf_emoji/model_checkpoint.data-00000-of-00001',
 '/content/model_prado_base_pdf_emoji/model_checkpoint.index']

In [ ]:
model_hate_eval = build_model(base_layers.EVAL)
metrics = create_metrics(LABELS)
model_hate_eval.compile(metrics=metrics)
model_hate_eval.load_weights('model_prado_base_pdf_emoji_quant/model_checkpoint')
result = model_hate_eval.evaluate(x=test_dataset_hate, return_dict=True)

prec = result['precision@0.5/hate']
rec = result['recall@0.5/all']
f1 = 2 * (prec*rec) / (prec+rec)
print(f1)

## word2vec + PRADO

In [ ]:
LABELS = [
    'hate',
]
CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,
}
CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)
MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': False,
    'quantize': False, # maybe True??
    'max_seq_len': 64,
    'max_seq_len_inference': 64,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 128,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,
}
CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
clean_train = clean_dataset_with_emoji(text)
clean_test = clean_dataset_with_emoji(text_test)


word2vec 100 old processing

In [ ]:
model_emb_100 = gensim.models.Word2Vec.load("word2vec2_100.model") # old processing

In [ ]:
clean_train = clean_dataset(texts_labels)
clean_test = clean_dataset(texts_labels_test)

train_hate_emb1 = create_dict_for_tf_emb(clean_train, Y_train, model_emb_100, emb_len=100, max_num_words_sen=128)
test_hate_emb1 = create_dict_for_tf_emb(clean_test, Y_test, model_emb_100, emb_len=100, max_num_words_sen=128)

train_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb1, test_hate_emb1)
test_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb1, test_hate_emb1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 128, 100)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 128, 100)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb1, test_dataset_hate_emb1, CONFIG, LABELS, 'model_emb_word2vec1_rerun')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 30s 17ms/step - loss: 0.6128 - accuracy: 0.6794
Epoch 2/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4966 - accuracy: 0.8304
Epoch 3/10
1000/1000 [==============================] - 18s 18ms/step - loss: 0.4660 - accuracy: 0.8694
Epoch 4/10
1000/1000 [==============================] - 18s 18ms/step - loss: 0.4552 - accuracy: 0.8812
Epoch 5/10
1000/1000 [==============================] - 18s 18ms/step - loss: 0.4519 - accuracy: 0.8836
Epoch 6/10
1000/1000 [==============================] - 18s 18ms/step - loss: 0.4501 - accuracy: 0.8881
Epoch 7/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4499 - accuracy: 0.8863
Epoch 8/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4500 - accuracy: 0.8865
Epo

word2vec 100 new processing

In [ ]:
model_emb_100 = gensim.models.Word2Vec.load("word2vec2_100_new_proc.model")

In [ ]:
clean_train = clean_dataset_with_emoji(text) #better processing no emoji
clean_test = clean_dataset_with_emoji(text_test)

train_hate_emb2 = create_dict_for_tf_emb(clean_train, Y_train, model_emb_100, emb_len=100, max_num_words_sen=128)
test_hate_emb2 = create_dict_for_tf_emb(clean_test, Y_test, model_emb_100, emb_len=100, max_num_words_sen=128)

train_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb2, test_hate_emb2)
test_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb2, test_hate_emb2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 128, 100)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 128, 100)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb1, test_dataset_hate_emb1, CONFIG, LABELS, 'model_emb_word2vec4_rerun')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 19s 17ms/step - loss: 0.6107 - accuracy: 0.6781
Epoch 2/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4953 - accuracy: 0.8326
Epoch 3/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4648 - accuracy: 0.8701
Epoch 4/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4542 - accuracy: 0.8809
Epoch 5/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4507 - accuracy: 0.8837
Epoch 6/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4491 - accuracy: 0.8848
Epoch 7/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4481 - accuracy: 0.8874
Epoch 8/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4486 - accuracy: 0.8852
Epo

In [ ]:
(39+782)/945

0.8687830687830688

word2vec 128 new processing

In [ ]:
model_emb_128 = gensim.models.Word2Vec.load("word2vec2_128_new_proc.model")

In [ ]:
clean_train = clean_dataset_with_emoji(text) #better processing no emoji
clean_test = clean_dataset_with_emoji(text_test)

train_hate_emb2 = create_dict_for_tf_emb(clean_train, Y_train, model_emb_128, emb_len=128, max_num_words_sen=128)
test_hate_emb2 = create_dict_for_tf_emb(clean_test, Y_test, model_emb_128, emb_len=128, max_num_words_sen=128)

train_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb2, test_hate_emb2)
test_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb2, test_hate_emb2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 128, 128)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 128, 128)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb1, test_dataset_hate_emb1, CONFIG, LABELS, 'model_emb_word2vec2_rerun')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 20s 18ms/step - loss: 0.6093 - accuracy: 0.6844
Epoch 2/10
1000/1000 [==============================] - 18s 18ms/step - loss: 0.4937 - accuracy: 0.8339
Epoch 3/10
1000/1000 [==============================] - 18s 18ms/step - loss: 0.4628 - accuracy: 0.8713
Epoch 4/10
1000/1000 [==============================] - 18s 18ms/step - loss: 0.4521 - accuracy: 0.8839
Epoch 5/10
1000/1000 [==============================] - 19s 19ms/step - loss: 0.4484 - accuracy: 0.8873
Epoch 6/10
1000/1000 [==============================] - 19s 19ms/step - loss: 0.4469 - accuracy: 0.8891
Epoch 7/10
1000/1000 [==============================] - 18s 18ms/step - loss: 0.4469 - accuracy: 0.8894
Epoch 8/10
1000/1000 [==============================] - 19s 19ms/step - loss: 0.4465 - accuracy: 0.8894
Epo

In [ ]:
(47+771)/945

0.8656084656084656

 word2vec 128  len64 

In [ ]:
clean_train = clean_dataset_with_emoji(text) #better processing no emoji
clean_test = clean_dataset_with_emoji(text_test)

In [ ]:
train_hate_emb2 = create_dict_for_tf_emb(clean_train, Y_train, model_emb_128, emb_len=128, max_num_words_sen=64)
test_hate_emb2 = create_dict_for_tf_emb(clean_test, Y_test, model_emb_128, emb_len=128, max_num_words_sen=64)

train_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb2, test_hate_emb2)
test_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb2, test_hate_emb2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 128)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 128)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb1, test_dataset_hate_emb1, CONFIG, LABELS, 'model_emb_word2vec8_rerun')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 134s 131ms/step - loss: 0.6105 - accuracy: 0.6800
Epoch 2/10
1000/1000 [==============================] - 131s 131ms/step - loss: 0.4945 - accuracy: 0.8336
Epoch 3/10
1000/1000 [==============================] - 132s 132ms/step - loss: 0.4625 - accuracy: 0.8727
Epoch 4/10
1000/1000 [==============================] - 136s 136ms/step - loss: 0.4527 - accuracy: 0.8832
Epoch 5/10
1000/1000 [==============================] - 131s 131ms/step - loss: 0.4490 - accuracy: 0.8865
Epoch 6/10
1000/1000 [==============================] - 130s 130ms/step - loss: 0.4476 - accuracy: 0.8885
Epoch 7/10
1000/1000 [==============================] - 131s 131ms/step - loss: 0.4469 - accuracy: 0.8890
Epoch 8/10
1000/1000 [==============================] - 135s 135ms/step - loss: 0.4473 - accu

word2vec 128 new processing skip gram

In [ ]:
model_emb_128 = gensim.models.Word2Vec.load("word2vec2_128_new_proc_sg.model")

In [ ]:
clean_train = clean_dataset_with_emoji(text) #better processing no emoji
clean_test = clean_dataset_with_emoji(text_test)

train_hate_emb2 = create_dict_for_tf_emb(clean_train, Y_train, model_emb_128, emb_len=128, max_num_words_sen=128)
test_hate_emb2 = create_dict_for_tf_emb(clean_test, Y_test, model_emb_128, emb_len=128, max_num_words_sen=128)

train_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb2, test_hate_emb2)
test_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb2, test_hate_emb2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 128, 128)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 128, 128)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb1, test_dataset_hate_emb1, CONFIG, LABELS, 'model_emb_word2vec3_rerun')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 20s 18ms/step - loss: 0.5909 - accuracy: 0.7443
Epoch 2/10
1000/1000 [==============================] - 18s 18ms/step - loss: 0.4775 - accuracy: 0.8680
Epoch 3/10
1000/1000 [==============================] - 19s 19ms/step - loss: 0.4466 - accuracy: 0.8899
Epoch 4/10
1000/1000 [==============================] - 19s 19ms/step - loss: 0.4361 - accuracy: 0.8972
Epoch 5/10
1000/1000 [==============================] - 18s 18ms/step - loss: 0.4323 - accuracy: 0.8995
Epoch 6/10
1000/1000 [==============================] - 18s 18ms/step - loss: 0.4309 - accuracy: 0.8999
Epoch 7/10
1000/1000 [==============================] - 18s 18ms/step - loss: 0.4302 - accuracy: 0.9008
Epoch 8/10
1000/1000 [==============================] - 19s 19ms/step - loss: 0.4307 - accuracy: 0.8993
Epo

In [ ]:
(63+779)/945

0.891005291005291

word2vec 128 len 64 sg 

In [ ]:
LABELS = [
    'hate',
]

CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,
}
CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)
MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': False,
    'quantize': False,
    'max_seq_len': 64,
    'max_seq_len_inference': 64,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 128,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,
}
CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
clean_train = clean_dataset_with_emoji(text) #better processing no emoji
clean_test = clean_dataset_with_emoji(text_test)
train_hate_emb2 = create_dict_for_tf_emb(clean_train, Y_train, model_emb_128, emb_len=128, max_num_words_sen=64)
test_hate_emb2 = create_dict_for_tf_emb(clean_test, Y_test, model_emb_128, emb_len=128, max_num_words_sen=64)

train_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb2, test_hate_emb2)
test_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb2, test_hate_emb2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 128)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 128)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb1, test_dataset_hate_emb1, CONFIG, LABELS, 'model_emb_word2vec9_rerun')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 132s 129ms/step - loss: 0.5914 - accuracy: 0.7446
Epoch 2/10
1000/1000 [==============================] - 129s 129ms/step - loss: 0.4795 - accuracy: 0.8660
Epoch 3/10
1000/1000 [==============================] - 133s 133ms/step - loss: 0.4481 - accuracy: 0.8900
Epoch 4/10
1000/1000 [==============================] - 129s 129ms/step - loss: 0.4378 - accuracy: 0.8962
Epoch 5/10
1000/1000 [==============================] - 128s 128ms/step - loss: 0.4342 - accuracy: 0.8982
Epoch 6/10
1000/1000 [==============================] - 129s 129ms/step - loss: 0.4322 - accuracy: 0.8988
Epoch 7/10
1000/1000 [==============================] - 132s 132ms/step - loss: 0.4321 - accuracy: 0.8998
Epoch 8/10
1000/1000 [==============================] - 129s 129ms/step - loss: 0.4325 - accu

In [ ]:
model_hate_eval2 = build_model(base_layers.EVAL)
metrics = create_metrics(LABELS)
model_hate_eval2.compile(metrics=metrics)
model_hate_eval2.load_weights('model_emb_word2vec9_rerun/model_checkpoint')
result = model_hate_eval2.evaluate(x=test_dataset_hate_emb1, return_dict=True)

prec = result['precision@0.5/hate']
rec = result['recall@0.5/all']
f1 = 2 * (prec*rec) / (prec+rec)
print(f1)

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")


9/9 [==============================] - 5s 20ms/step - loss: 0.0000e+00 - precision@0.5/hate: 0.5935 - recall@0.5/hate: 0.6033 - TruePositives@0.5/hate: 73.0000 - TrueNegatives@0.5/hate: 774.0000 - FalsePositives@0.5/hate: 50.0000 - FalseNegatives@0.5/hate: 48.0000 - precision@0.5/all: 0.5935 - recall@0.5/all: 0.6033
0.5983606269044986


In [ ]:
# save_classification_results(model_hate_eval2, test_dataset_hate_emb1, clean_test, Y_test, 'word2vec_best')

In [ ]:
with open('word2vec_best.pickle', 'rb') as handle:
    word2vec_best = pickle.load(handle)

In [ ]:
(73+774)/945

0.8962962962962963

word2vec 256 sg

In [ ]:
LABELS = [
    'hate',
]

CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,
}
CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)
MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': False,
    'quantize': False,
    'max_seq_len': 128,
    'max_seq_len_inference': 128,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 256,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,
}
CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
del model_emb_256

In [ ]:
model_emb_256 = gensim.models.Word2Vec.load("word2vec2_256_new_proc_sg.model")

In [ ]:
clean_train = clean_dataset_with_emoji(text) #better processing no emoji
clean_test = clean_dataset_with_emoji(text_test)

train_hate_emb2 = create_dict_for_tf_emb(clean_train, Y_train, model_emb_256, emb_len=256, max_num_words_sen=128)
test_hate_emb2 = create_dict_for_tf_emb(clean_test, Y_test, model_emb_256, emb_len=256, max_num_words_sen=128)

train_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb2, test_hate_emb2)
test_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb2, test_hate_emb2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 128, 256)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 128, 256)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb1, test_dataset_hate_emb1, CONFIG, LABELS, 'model_emb_word2vec5_rerun')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 39s 25ms/step - loss: 0.5848 - accuracy: 0.7582
Epoch 2/10
1000/1000 [==============================] - 25s 25ms/step - loss: 0.4696 - accuracy: 0.8770
Epoch 3/10
1000/1000 [==============================] - 25s 25ms/step - loss: 0.4383 - accuracy: 0.8986
Epoch 4/10
1000/1000 [==============================] - 25s 25ms/step - loss: 0.4271 - accuracy: 0.9046
Epoch 5/10
1000/1000 [==============================] - 25s 25ms/step - loss: 0.4225 - accuracy: 0.9075
Epoch 6/10
1000/1000 [==============================] - 25s 25ms/step - loss: 0.4215 - accuracy: 0.9092
Epoch 7/10
1000/1000 [==============================] - 25s 25ms/step - loss: 0.4215 - accuracy: 0.9091
Epoch 8/10
1000/1000 [==============================] - 25s 25ms/step - loss: 0.4211 - accuracy: 0.9087
Epo

In [ ]:
(62+787)/945

0.8984126984126984

word2vec 256 sg sen len 32

In [ ]:
LABELS = [
    'hate',
]

CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,
}
CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)
MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': False,
    'quantize': False,
    'max_seq_len': 32,
    'max_seq_len_inference': 32,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 256,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,
}
CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
clean_train = clean_dataset_with_emoji(text) #better processing no emoji
clean_test = clean_dataset_with_emoji(text_test)

train_hate_emb2 = create_dict_for_tf_emb(clean_train, Y_train, model_emb_256, emb_len=256, max_num_words_sen=32)
test_hate_emb2 = create_dict_for_tf_emb(clean_test, Y_test, model_emb_256, emb_len=256, max_num_words_sen=32)

train_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb2, test_hate_emb2)
test_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb2, test_hate_emb2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 32, 256)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 32, 256)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb1, test_dataset_hate_emb1, CONFIG, LABELS, 'model_emb_word2vec6_rerun')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 13s 11ms/step - loss: 0.5836 - accuracy: 0.7581
Epoch 2/10
1000/1000 [==============================] - 11s 11ms/step - loss: 0.4689 - accuracy: 0.8771
Epoch 3/10
1000/1000 [==============================] - 11s 11ms/step - loss: 0.4380 - accuracy: 0.8980
Epoch 4/10
1000/1000 [==============================] - 11s 11ms/step - loss: 0.4274 - accuracy: 0.9038
Epoch 5/10
1000/1000 [==============================] - 11s 11ms/step - loss: 0.4222 - accuracy: 0.9090
Epoch 6/10
1000/1000 [==============================] - 11s 11ms/step - loss: 0.4212 - accuracy: 0.9081
Epoch 7/10
1000/1000 [==============================] - 12s 12ms/step - loss: 0.4202 - accuracy: 0.9100
Epoch 8/10
1000/1000 [==============================] - 11s 11ms/step - loss: 0.4204 - accuracy: 0.9088
Epo

word2vec 256 sg sen len64

In [ ]:
LABELS = [
    'hate',
]

CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,
}
CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)
MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': False,
    'quantize': False,
    'max_seq_len': 64,
    'max_seq_len_inference': 64,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 256,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,
}
CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
clean_train = clean_dataset_with_emoji(text) #better processing no emoji
clean_test = clean_dataset_with_emoji(text_test)

train_hate_emb2 = create_dict_for_tf_emb(clean_train, Y_train, model_emb_256, emb_len=256, max_num_words_sen=64)
test_hate_emb2 = create_dict_for_tf_emb(clean_test, Y_test, model_emb_256, emb_len=256, max_num_words_sen=64)

train_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb2, test_hate_emb2)
test_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb2, test_hate_emb2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 256)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 256)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb1, test_dataset_hate_emb1, CONFIG, LABELS, 'model_emb_word2vec7_rerun')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 17s 15ms/step - loss: 0.5828 - accuracy: 0.7637
Epoch 2/10
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4700 - accuracy: 0.8758
Epoch 3/10
1000/1000 [==============================] - 15s 15ms/step - loss: 0.4377 - accuracy: 0.8992
Epoch 4/10
1000/1000 [==============================] - 15s 15ms/step - loss: 0.4275 - accuracy: 0.9047
Epoch 5/10
1000/1000 [==============================] - 15s 15ms/step - loss: 0.4229 - accuracy: 0.9076
Epoch 6/10
1000/1000 [==============================] - 15s 15ms/step - loss: 0.4220 - accuracy: 0.9080
Epoch 7/10
1000/1000 [==============================] - 15s 15ms/step - loss: 0.4212 - accuracy: 0.9081
Epoch 8/10
1000/1000 [==============================] - 15s 15ms/step - loss: 0.4213 - accuracy: 0.9086
Epo

In [ ]:
(71+776)/945

0.8962962962962963

### word2vec 128 with train and emoji

In [ ]:
LABELS = [
    'hate',
]

# Model training parameters.
CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,
}


CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)

MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': False,
    'quantize': False, 
    'max_seq_len': 64,
    'max_seq_len_inference': 64,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 128,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,

}

CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
# word2vec embeddings
!gdown --id 1--XWguD6pIHgNzZrnJMSAQAk8hDL772C
!gdown --id 1TOEjm107vxBMGo_PQy3M9dqrsDgWFuJf
!gdown --id 1tgeJYpHXjUesrZzeswU_1VD2D1oO5uk5


In [ ]:
model_emb_128 = gensim.models.Word2Vec.load("word2vec_128_rerun_emoji_train.model")

In [ ]:
clean_train = clean_dataset_with_emoji(text, keep_emoji=True) #better processing no emoji
clean_test = clean_dataset_with_emoji(text_test, keep_emoji=True)

train_hate_emb2 = create_dict_for_tf_emb(clean_train, Y_train, model_emb_128, emb_len=128, max_num_words_sen=64)
test_hate_emb2 = create_dict_for_tf_emb(clean_test, Y_test, model_emb_128, emb_len=128, max_num_words_sen=64)

train_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb2, test_hate_emb2)
test_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb2, test_hate_emb2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 128)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 128)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb1, test_dataset_hate_emb1, CONFIG, LABELS, 'model_emb_word2vec10_rerun')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 139s 136ms/step - loss: 0.5892 - accuracy: 0.7518
Epoch 2/10
1000/1000 [==============================] - 132s 132ms/step - loss: 0.4778 - accuracy: 0.8671
Epoch 3/10
1000/1000 [==============================] - 138s 138ms/step - loss: 0.4469 - accuracy: 0.8900
Epoch 4/10
1000/1000 [==============================] - 129s 129ms/step - loss: 0.4369 - accuracy: 0.8960
Epoch 5/10
1000/1000 [==============================] - 130s 130ms/step - loss: 0.4323 - accuracy: 0.8992
Epoch 6/10
1000/1000 [==============================] - 130s 130ms/step - loss: 0.4311 - accuracy: 0.9003
Epoch 7/10
1000/1000 [==============================] - 130s 130ms/step - loss: 0.4311 - accuracy: 0.8989
Epoch 8/10
1000/1000 [==============================] - 131s 131ms/step - loss: 0.4317 - accu

In [12]:
gdown.download_folder('https://drive.google.com/drive/folders/1nAfTIN7ppvThQyiyz2DiwqN4Hhv8fyPQ?usp=sharing', quiet=True)

['/content/model_emb_word2vec10_rerun/checkpoint',
 '/content/model_emb_word2vec10_rerun/model_checkpoint.data-00000-of-00001',
 '/content/model_emb_word2vec10_rerun/model_checkpoint.index']

In [ ]:
model_hate_eval = build_model(base_layers.EVAL)
metrics = create_metrics(LABELS)
model_hate_eval.compile(metrics=metrics)
model_hate_eval.load_weights('model_emb_word2vec10_rerun/model_checkpoint')
result = model_hate_eval.evaluate(x=test_dataset_hate, return_dict=True)

prec = result['precision@0.5/hate']
rec = result['recall@0.5/all']
f1 = 2 * (prec*rec) / (prec+rec)
print(f1)

In [ ]:
(71+769)/945

0.8888888888888888

### compare prado and word2vec

In [ ]:
with open('word2vec_best.pickle', 'rb') as handle:
    word2bec_best = pickle.load(handle)

with open('prado_best.pickle', 'rb') as handle:
    prado_best = pickle.load(handle)

In [ ]:
for item in prado_best['tp']:
  if item not in word2bec_best['tp']:
    print(item)
print()
for item in word2bec_best['tp']:
  if item not in prado_best['tp']:
    print(item)

ten szmat się nie komentować
dokładnie pisdzielstwo nie mój prawa rozpierdalać system sądownictwo nie maić większość
półgłówek wieliński wymyślić soba półautorytaryzm
podstawowy zadać każdy ksiądz to narzucać wiara
rozumieć że standardowy polski rodzina to pierdolnąć stary kawaler zoofil i kot
gdyby być u władza spałować taki marsz
on być na poziom pierdolnąć zaawansowany więc nie dziwić nic
z trolami nie dyskutować a z głupi zwłaszcza
ty załgany łachudra jakby nie brejza to nikt by o tym nie wiedzieć
i tak dobrnąć na dno szambo
hehe do gaz hehe marzec 2018
a może co o pmm który sprowokować zachować się jak typowy pisowski nieuk i idiota
wysoki porać zaistnieć w 21 wiek a wy odizolować od zdrowy częsty społeczeństwo o nie pozwolić na chory indoktrynacja
jezdzisz na wózek tramwaj uciał ci noga
i tym sposób zwyciężczyni turniej o plastusiowego debil zostać agnieszka holland brawo uśmiech
ale ty być jebnięty rzadko spotykać egzemplarz nie uwarzasz że im to wisieć
podobno przed sejm ustawi

In [ ]:
curr_cat = 'fn'
for item in prado_best[curr_cat]:
  if item not in word2bec_best[curr_cat]:
    print(item)
print()
for item in word2bec_best[curr_cat]:
  if item not in prado_best[curr_cat]:
    print(item)

widać człowiek w sącz tak pasować ja by mu w gęba napluć
i wojna gadowski z psychiatra
chcieć aby tak być miernota
w powietrze sa środki chemiczny uważać wygladasz na podatny
tym który nie być na mecz z wzgląd na koniec zgoda mamić kilka słowić do przekazać wypierdalać do wrocław alba gdańska
rozjebać kc raz z i on ojciec tylko że wy to się tylko śnić
złodziejstwo kurestwo patologia kompleks brak pewność siebie
gdzie być to cały zakłamany lewacki skurwysynstwo no gdzie
ja dalej dziwić ale ciągły wasze ujadać skamleć donosić i kablować na polska kretyn
twój chłop się jeszcze nie powiesić ja by już dyndać jak by miał taki idiotka
zrzec się polskie obywatelstwo dla karier pogratulować
jaki tama profesor to kangur który aborygen zamienić w człowiek
ola wiedzieć że chodzić do burdel nadawać pan w kategoria zjebać rok 2018 pierwszy miejsce
jaka obywatel kategorycznie domagać się aby włożyć soba nos w końcowy odcinek układ pokarmowy
piskomuna zdychać w lęk nnasze słowo on udręka
być tym z wod

## fasttext + PRADO

fasttext 100 sg

In [ ]:
LABELS = [
    'hate',
]

CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,
}

CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)

MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': False,
    'quantize': False, 
    'max_seq_len': 128,
    'max_seq_len_inference': 128,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 128,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,
}

CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
model_emb_100 = gensim.models.Word2Vec.load("word2vec_fast_100_rerun.model")

In [ ]:
clean_train = clean_dataset_with_emoji(text) #better processing no emoji
clean_test = clean_dataset_with_emoji(text_test)

train_hate_emb2 = create_dict_for_tf_emb(clean_train, Y_train, model_emb_100, emb_len=100, max_num_words_sen=128)
test_hate_emb2 = create_dict_for_tf_emb(clean_test, Y_test, model_emb_100, emb_len=100, max_num_words_sen=128)

train_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb2, test_hate_emb2)
test_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb2, test_hate_emb2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 128, 100)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 128, 100)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb1, test_dataset_hate_emb1, CONFIG, LABELS, 'model_emb_fast1_rerun')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 21s 19ms/step - loss: 0.5908 - accuracy: 0.7460
Epoch 2/10
1000/1000 [==============================] - 18s 18ms/step - loss: 0.4791 - accuracy: 0.8645
Epoch 3/10
1000/1000 [==============================] - 19s 19ms/step - loss: 0.4479 - accuracy: 0.8892
Epoch 4/10
1000/1000 [==============================] - 18s 18ms/step - loss: 0.4382 - accuracy: 0.8944
Epoch 5/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4338 - accuracy: 0.8977
Epoch 6/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4326 - accuracy: 0.8981
Epoch 7/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4323 - accuracy: 0.8973
Epoch 8/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4325 - accuracy: 0.8978
Epo

In [ ]:
(60+783)/945

0.8920634920634921

fast 128 sg

In [ ]:
model_emb_100 = gensim.models.Word2Vec.load("word2vec_fast_128_rerun.model")

In [ ]:
clean_train = clean_dataset_with_emoji(text) #better processing no emoji
clean_test = clean_dataset_with_emoji(text_test)

train_hate_emb2 = create_dict_for_tf_emb(clean_train, Y_train, model_emb_100, emb_len=128, max_num_words_sen=128)
test_hate_emb2 = create_dict_for_tf_emb(clean_test, Y_test, model_emb_100, emb_len=128, max_num_words_sen=128)

train_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb2, test_hate_emb2)
test_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb2, test_hate_emb2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 128, 128)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 128, 128)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb1, test_dataset_hate_emb1, CONFIG, LABELS, 'model_emb_fast2_rerun')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 32s 19ms/step - loss: 0.5900 - accuracy: 0.7472
Epoch 2/10
1000/1000 [==============================] - 19s 19ms/step - loss: 0.4784 - accuracy: 0.8644
Epoch 3/10
1000/1000 [==============================] - 19s 19ms/step - loss: 0.4476 - accuracy: 0.8881
Epoch 4/10
1000/1000 [==============================] - 19s 19ms/step - loss: 0.4378 - accuracy: 0.8950
Epoch 5/10
1000/1000 [==============================] - 19s 19ms/step - loss: 0.4335 - accuracy: 0.8970
Epoch 6/10
1000/1000 [==============================] - 19s 19ms/step - loss: 0.4323 - accuracy: 0.8971
Epoch 7/10
1000/1000 [==============================] - 20s 20ms/step - loss: 0.4315 - accuracy: 0.8975
Epoch 8/10
1000/1000 [==============================] - 19s 19ms/step - loss: 0.4313 - accuracy: 0.8986
Epo

In [ ]:
(63+778)/945

0.8899470899470899

fast 128 sg sen len 64

In [ ]:
LABELS = [
    'hate',
]

CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,
}

CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)

MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': False,
    'quantize': False, 
    'max_seq_len': 64,
    'max_seq_len_inference': 64,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 128,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,
}

CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
model_emb_100 = gensim.models.Word2Vec.load("word2vec_fast_128_rerun.model")

In [ ]:
clean_train = clean_dataset_with_emoji(text) #better processing no emoji
clean_test = clean_dataset_with_emoji(text_test)

train_hate_emb2 = create_dict_for_tf_emb(clean_train, Y_train, model_emb_100, emb_len=128, max_num_words_sen=64)
test_hate_emb2 = create_dict_for_tf_emb(clean_test, Y_test, model_emb_100, emb_len=128, max_num_words_sen=64)

train_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb2, test_hate_emb2)
test_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb2, test_hate_emb2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 128)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 128)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb1, test_dataset_hate_emb1, CONFIG, LABELS, 'model_emb_fast3_rerun')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 129s 126ms/step - loss: 0.5880 - accuracy: 0.7531
Epoch 2/10
1000/1000 [==============================] - 127s 127ms/step - loss: 0.4771 - accuracy: 0.8666
Epoch 3/10
1000/1000 [==============================] - 125s 125ms/step - loss: 0.4458 - accuracy: 0.8899
Epoch 4/10
1000/1000 [==============================] - 126s 126ms/step - loss: 0.4356 - accuracy: 0.8968
Epoch 5/10
1000/1000 [==============================] - 125s 125ms/step - loss: 0.4321 - accuracy: 0.8986
Epoch 6/10
1000/1000 [==============================] - 125s 125ms/step - loss: 0.4309 - accuracy: 0.9001
Epoch 7/10
1000/1000 [==============================] - 127s 127ms/step - loss: 0.4303 - accuracy: 0.8990
Epoch 8/10
1000/1000 [==============================] - 126s 126ms/step - loss: 0.4304 - accu

In [ ]:
(59+794)/945

0.9026455026455027

fast text with train data

In [ ]:
del model_emb_100

In [ ]:
model_emb_100 = gensim.models.Word2Vec.load("word2vec_fast_128_with_train_rerun.model")

In [ ]:
clean_train = clean_dataset_with_emoji(text) #better processing no emoji
clean_test = clean_dataset_with_emoji(text_test)

train_hate_emb2 = create_dict_for_tf_emb(clean_train, Y_train, model_emb_100, emb_len=128, max_num_words_sen=64)
test_hate_emb2 = create_dict_for_tf_emb(clean_test, Y_test, model_emb_100, emb_len=128, max_num_words_sen=64)

train_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb2, test_hate_emb2)
test_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb2, test_hate_emb2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 128)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 128)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb1, test_dataset_hate_emb1, CONFIG, LABELS, 'model_emb_fast4_rerun')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 128s 126ms/step - loss: 0.5868 - accuracy: 0.7558
Epoch 2/10
1000/1000 [==============================] - 128s 128ms/step - loss: 0.4763 - accuracy: 0.8669
Epoch 3/10
1000/1000 [==============================] - 126s 126ms/step - loss: 0.4458 - accuracy: 0.8909
Epoch 4/10
1000/1000 [==============================] - 127s 127ms/step - loss: 0.4342 - accuracy: 0.8978
Epoch 5/10
1000/1000 [==============================] - 127s 127ms/step - loss: 0.4309 - accuracy: 0.8993
Epoch 6/10
1000/1000 [==============================] - 127s 127ms/step - loss: 0.4298 - accuracy: 0.9004
Epoch 7/10
1000/1000 [==============================] - 127s 127ms/step - loss: 0.4296 - accuracy: 0.9007
Epoch 8/10
1000/1000 [==============================] - 128s 128ms/step - loss: 0.4297 - accu

In [ ]:
(65+776)/945

0.8899470899470899

### fast text with emoji and train

In [ ]:
LABELS = [
    'hate',
]
CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,
}
CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)
MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': False,
    'quantize': False, 
    'max_seq_len': 64,
    'max_seq_len_inference': 64,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 128,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,
}
CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
del model_emb_100

In [ ]:
# fasttext embeddings
!gdown --id 1-NEUypQa1hg9FTWORkC12i7K8S1Vr4dC
!gdown --id 1-O8JI7QHQftkRlCzpdM8Wrgfh1qlpsQ6
!gdown --id 1-OXeyORmr9PcQWR0HDVFcKJ40Pwv31P9
!gdown --id 1-X2VCJJ7NE1bWblaXlf_Vo9CLHwJ1gmC
!gdown --id 1-YDmeuyYEntxIEteJMoBKl3fFmz408lH
!gdown --id 1-e7ny1JOo8OROVLb6a64gPAUMSV2U2F5
!gdown --id 1-eAtiuWmM2M0AHGjoBlcUb_YHqBeCuRh


In [ ]:
model_emb_100 = gensim.models.Word2Vec.load("word2vec_fast_128_with_train__emoji_rerun.model")

In [ ]:
clean_train = clean_dataset_with_emoji(text, keep_emoji=True)
clean_test = clean_dataset_with_emoji(text_test, keep_emoji=True)

train_hate_emb2 = create_dict_for_tf_emb(clean_train, Y_train, model_emb_100, emb_len=128, max_num_words_sen=64)
test_hate_emb2 = create_dict_for_tf_emb(clean_test, Y_test, model_emb_100, emb_len=128, max_num_words_sen=64)

train_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb2, test_hate_emb2)
test_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb2, test_hate_emb2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 128)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 128)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb1, test_dataset_hate_emb1, CONFIG, LABELS, 'model_emb_fast5_rerun')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 132s 129ms/step - loss: 0.5897 - accuracy: 0.7491
Epoch 2/10
1000/1000 [==============================] - 130s 130ms/step - loss: 0.4785 - accuracy: 0.8665
Epoch 3/10
1000/1000 [==============================] - 129s 129ms/step - loss: 0.4470 - accuracy: 0.8885
Epoch 4/10
1000/1000 [==============================] - 131s 131ms/step - loss: 0.4365 - accuracy: 0.8953
Epoch 5/10
1000/1000 [==============================] - 130s 130ms/step - loss: 0.4331 - accuracy: 0.8985
Epoch 6/10
1000/1000 [==============================] - 132s 132ms/step - loss: 0.4316 - accuracy: 0.8981
Epoch 7/10
1000/1000 [==============================] - 131s 131ms/step - loss: 0.4306 - accuracy: 0.8999
Epoch 8/10
1000/1000 [==============================] - 131s 131ms/step - loss: 0.4311 - accu

In [ ]:
(72+777)/945

0.8984126984126984

In [ ]:
model_hate_eval2 = build_model(base_layers.EVAL)
metrics = create_metrics(LABELS)
model_hate_eval2.compile(metrics=metrics)
model_hate_eval2.load_weights('model_emb_fast5_rerun/model_checkpoint')
result = model_hate_eval2.evaluate(x=test_dataset_hate_emb1, return_dict=True)

prec = result['precision@0.5/hate']
rec = result['recall@0.5/all']
f1 = 2 * (prec*rec) / (prec+rec)
print(f1)

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
9/9 [==============================] - 1s 17ms/step - loss: 0.0000e+00 - precision@0.5/hate: 0.6050 - recall@0.5/hate: 0.5950 - TruePositives@0.5/hate: 72.0000 - TrueNegatives@0.5/hate: 777.0000 - FalsePositives@0.5/hate: 47.0000 - FalseNegatives@0.5/hate: 49.0000 - precision@0.5/all: 0.6050 - recall@0.5/all: 0.5950
0.6000000178358621


In [ ]:
save_classification_results(model_hate_eval2, test_dataset_hate_emb1, clean_test, Y_test, 'fasttext_best')

In [13]:
gdown.download_folder('https://drive.google.com/drive/folders/1-uysb0dZfX9V-v9ro5Y5eNnyyj9M3a_A?usp=sharing', quiet=True)

['/content/model_emb_fast5_rerun/checkpoint',
 '/content/model_emb_fast5_rerun/model_checkpoint.data-00000-of-00001',
 '/content/model_emb_fast5_rerun/model_checkpoint.index']

In [ ]:
model_hate_eval = build_model(base_layers.EVAL)
metrics = create_metrics(LABELS)
model_hate_eval.compile(metrics=metrics)
model_hate_eval.load_weights('model_emb_fast5_rerun/model_checkpoint')
result = model_hate_eval.evaluate(x=test_dataset_hate, return_dict=True)

prec = result['precision@0.5/hate']
rec = result['recall@0.5/all']
f1 = 2 * (prec*rec) / (prec+rec)
print(f1)

### compare word2vec and fasttext

In [ ]:
with open('word2vec_best.pickle', 'rb') as handle:
    word2bec_best = pickle.load(handle)

with open('fasttext_best.pickle', 'rb') as handle:
    fasttext_best = pickle.load(handle)

In [ ]:
for item in fasttext_best['tp']:
  if item not in word2bec_best['tp']:
    print(item)
print()
for item in word2bec_best['tp']:
  if item not in fasttext_best['tp']:
    print(item)

dokładnie pisdzielstwo nie mój prawa rozpierdalać system sądownictwo nie maić większość
rozumieć że standardowy polski rodzina to pierdolnąć stary kawaler zoofil i kot
gdzie być to cały zakłamany lewacki skurwysynstwo no gdzie
jaki tama profesor to kangur który aborygen zamienić w człowiek
on być na poziom pierdolnąć zaawansowany więc nie dziwić nic
ten pierwszy pójść na wojna być my bronić patriota nawet miska ryż nie chcieć
piskomuna zdychać w lęk nnasze słowo on udręka
ty załgany łachudra jakby nie brejza to nikt by o tym nie wiedzieć
i tak dobrnąć na dno szambo
to pieprzyć komunał pad że być warta budzić w ja niesmak drugi tusk qrwa
a może co o pmm który sprowokować zachować się jak typowy pisowski nieuk i idiota
i tym sposób zwyciężczyni turniej o plastusiowego debil zostać agnieszka holland brawo uśmiech
trzymać bo on postraszyć że jak nie to być on tuczyć jak gęś
podobno przed sejm ustawić gilotyna maja obcinać ręka ubywatelom rp ręka który podnieść na władza
bankster od frankow

## test multi class problem

here further debugging is needed

In [ ]:
def create_dict_for_tf_emb_multi(texts, labels, embeddings, emb_len=100, max_num_words_sen=128, glove=False):
  labels_dict = ['text', 'hate', 'cyber']
  list_dict_to_tensor = []

  for text, lab in zip(texts, labels):
    value_hate = False
    value_cyber = False
    if lab == '1':
      value_cyber = True
    elif lab == '2':
      value_hate = True
    words = text.split()
    word_array = np.zeros([max_num_words_sen,emb_len])
    if glove:
      for i, word in enumerate(words):
        if word in embeddings.dictionary:
          word_array[i] = embeddings.word_vectors[embeddings.dictionary[word]]    
    else:
      for i , word in enumerate(words):
        if word in embeddings:
          word_array[i] = embeddings[word]  

    list_dict_to_tensor.append({'text':word_array, 'seq_len': len(words), 'hate':value_hate, 'cyber':value_cyber})
  # return list_dict_to_tensor
  return tf.data.Dataset.from_tensor_slices(pd.DataFrame.from_dict(list_dict_to_tensor).to_dict(orient="list"))

In [ ]:
del model_emb_100

In [ ]:
LABELS = [
    'cyber', 'hate',
]

CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,
}

CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)

MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': True,
    'quantize': False, 
    'max_seq_len': 64,
    'max_seq_len_inference': 64,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 512,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,

}

CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
model_emb_100 = gensim.models.Word2Vec.load("word2vec_fast_128_with_train__emoji_rerun.model")

In [ ]:
labels_test = read_file('test_set_only_tags_multi.txt')
labels_train = read_file('training_set_clean_only_tags_multi.txt')

texts_labels_train_multi = merge_lists(text, labels_train)
texts_labels_test_multi = merge_lists(text_test, labels_test)
Y_train_multi = [line[1] for line in texts_labels_train_multi if line[0][:2] != 'RT']
Y_test_multi = [line[1] for line in texts_labels_test_multi if line[0][:2] != 'RT']

In [ ]:
len(test_hate_emb2)

945

In [ ]:
train_hate_emb2 = create_dict_for_tf_emb_multi(clean_train, Y_train_multi, model_emb_100, emb_len=128, max_num_words_sen=64)
test_hate_emb2 = create_dict_for_tf_emb_multi(clean_test, Y_test_multi, model_emb_100, emb_len=128, max_num_words_sen=64)

train_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb2, test_hate_emb2)
test_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb2, test_hate_emb2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 128)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 128)
(105,)


In [ ]:
build_run_save_eval_model(train_dataset_hate_emb1, test_dataset_hate_emb1, CONFIG, LABELS, 'model_emb_fast1_rerun_multi')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 30s 17ms/step - loss: 0.6209 - accuracy: 0.5329
Epoch 2/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4942 - accuracy: 0.5543
Epoch 3/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4587 - accuracy: 0.5604
Epoch 4/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4467 - accuracy: 0.5619
Epoch 5/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4425 - accuracy: 0.5640
Epoch 6/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4411 - accuracy: 0.5634
Epoch 7/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4404 - accuracy: 0.5653
Epoch 8/10
1000/1000 [==============================] - 17s 17ms/step - loss: 0.4405 - accuracy: 0.5635
Epo

In [ ]:
with open('lsh_dict1_emoji_rerun.pickle', 'rb') as handle:
    lsh_dict1 = pickle.load(handle)

In [ ]:
clean_train = clean_dataset_with_emoji(text, keep_emoji=True) #better processing no emoji
clean_test = clean_dataset_with_emoji(text_test,keep_emoji=True)

train_hate_emb2 = create_dict_for_tf_emb_multi(clean_train, Y_train_multi, lsh_dict1, emb_len=512, max_num_words_sen=64)
test_hate_emb2 = create_dict_for_tf_emb_multi(clean_test, Y_test_multi, lsh_dict1, emb_len=512, max_num_words_sen=64)

train_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb2, test_hate_emb2)
test_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb2, test_hate_emb2)

105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 512)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 512)
(105,)


In [ ]:
build_run_save_eval_model(train_dataset_hate_emb1, test_dataset_hate_emb1, CONFIG, LABELS, 'model_emb_lsh_multi')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 168s 166ms/step - loss: 0.6155 - accuracy: 0.5449
Epoch 2/10
1000/1000 [==============================] - 173s 173ms/step - loss: 0.4911 - accuracy: 0.5678
Epoch 3/10
1000/1000 [==============================] - 171s 171ms/step - loss: 0.4535 - accuracy: 0.5725
Epoch 4/10
1000/1000 [==============================] - 168s 168ms/step - loss: 0.4412 - accuracy: 0.5783
Epoch 5/10
1000/1000 [==============================] - 167s 167ms/step - loss: 0.4365 - accuracy: 0.5803
Epoch 6/10
1000/1000 [==============================] - 166s 166ms/step - loss: 0.4349 - accuracy: 0.5808
Epoch 7/10
1000/1000 [==============================] - 168s 168ms/step - loss: 0.4347 - accuracy: 0.5794
Epoch 8/10
1000/1000 [==============================] - 166s 166ms/step - loss: 0.4346 - accu

## LSH + PRADO

In [ ]:
LABELS = ['hate',]
CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,}
CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)
MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': False,
    'quantize': False, # maybe True??
    'max_seq_len': 32,
    'max_seq_len_inference': 32,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 512,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,}
CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
!gdown --id 1htDMrlhCyMLsCsejBoTsoiELuuodoCJb # lsh_dict1

In [ ]:
with open('lsh_dict1.pickle', 'rb') as handle:
    lsh_dict1 = pickle.load(handle)

In [ ]:
train_hate_emb18 = create_dict_for_tf_emb(clean_train, Y_train, lsh_dict1, emb_len=512, max_num_words_sen=32)
test_hate_emb18 = create_dict_for_tf_emb(clean_test, Y_test, lsh_dict1, emb_len=512, max_num_words_sen=32)

train_dataset_hate_emb18 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb18, test_hate_emb18)
test_dataset_hate_emb18 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb18, test_hate_emb18)

105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 32, 512)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 32, 512)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb18, test_dataset_hate_emb18, CONFIG, LABELS, 'model_emb18')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 103s 101ms/step - loss: 0.5825 - accuracy: 0.7674
Epoch 2/10
1000/1000 [==============================] - 101s 101ms/step - loss: 0.4650 - accuracy: 0.8835
Epoch 3/10
1000/1000 [==============================] - 103s 103ms/step - loss: 0.4311 - accuracy: 0.9079
Epoch 4/10
1000/1000 [==============================] - 104s 104ms/step - loss: 0.4184 - accuracy: 0.9152
Epoch 5/10
1000/1000 [==============================] - 105s 105ms/step - loss: 0.4145 - accuracy: 0.9170
Epoch 6/10
1000/1000 [==============================] - 104s 104ms/step - loss: 0.4132 - accuracy: 0.9185
Epoch 7/10
1000/1000 [==============================] - 104s 104ms/step - loss: 0.4114 - accuracy: 0.9206
Epoch 8/10
1000/1000 [==============================] - 103s 103ms/step - loss: 0.4127 - accu

In [ ]:
# model_pred16 = build_model(base_layers.EVAL)
# metrics = create_metrics(LABELS)
# model_pred16.compile(metrics=metrics)
# model_pred16.load_weights(f'model_emb18/model_checkpoint')
# result = model_pred16.evaluate(x=test_dataset_hate_emb18, return_dict=True)
# prec = result['precision@0.5/hate']
# rec = result['recall@0.5/all']
# f1 = 2 * (prec*rec) / (prec+rec)
# print(f1)

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
9/9 [==============================] - 1s 32ms/step - loss: 0.0000e+00 - precision@0.5/hate: 0.6517 - recall@0.5/hate: 0.4793 - TruePositives@0.5/hate: 58.0000 - TrueNegatives@0.5/hate: 793.0000 - FalsePositives@0.5/hate: 31.0000 - FalseNegatives@0.5/hate: 63.0000 - precision@0.5/all: 0.6517 - recall@0.5/all: 0.4793
0.5523809684877222


In [5]:
gdown.download_folder('https://drive.google.com/drive/folders/1Al19tV79l0cdXalnf-z05Ql4IDj31aZG?usp=sharing', quiet=True)

['/content/model_emb18/checkpoint',
 '/content/model_emb18/model_checkpoint.data-00000-of-00001',
 '/content/model_emb18/model_checkpoint.index']

In [ ]:
model_hate_eval = build_model(base_layers.EVAL)
metrics = create_metrics(LABELS)
model_hate_eval.compile(metrics=metrics)
model_hate_eval.load_weights('model_emb18/model_checkpoint')
result = model_hate_eval.evaluate(x=test_dataset_hate, return_dict=True)

prec = result['precision@0.5/hate']
rec = result['recall@0.5/all']
f1 = 2 * (prec*rec) / (prec+rec)
print(f1)

 lsh with 3 values 512

In [ ]:
!gdown --id 1GKTjF9_qcsKFkQnd9sCIeACv2YfYzYhR # lsh_dict_3val512

In [ ]:
with open('lsh_dict_3val512.pickle', 'rb') as handle:
    lsh_dict1 = pickle.load(handle)

In [ ]:
train_hate_emb25 = create_dict_for_tf_emb(clean_train, Y_train, lsh_dict1, emb_len=512, max_num_words_sen=32)
test_hate_emb25 = create_dict_for_tf_emb(clean_test, Y_test, lsh_dict1, emb_len=512, max_num_words_sen=32)

train_dataset_hate_emb25 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb25, test_hate_emb25)
test_dataset_hate_emb25 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb25, test_hate_emb25)

105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 32, 512)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 32, 512)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb25, test_dataset_hate_emb25, CONFIG, LABELS, 'model_emb25')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 28s 15ms/step - loss: 0.5829 - accuracy: 0.7687
Epoch 2/10
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4697 - accuracy: 0.8759
Epoch 3/10
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4355 - accuracy: 0.9030
Epoch 4/10
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4243 - accuracy: 0.9098
Epoch 5/10
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4197 - accuracy: 0.9126
Epoch 6/10
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4184 - accuracy: 0.9122
Epoch 7/10
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4178 - accuracy: 0.9143
Epoch 8/10
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4177 - accuracy: 0.9130
Epo

In [4]:
gdown.download_folder('https://drive.google.com/drive/folders/1NB_t5JfXTT2ATBne5aJv8yhJk88Zwm4l?usp=sharing', quiet=True)

['/content/model_emb25/checkpoint',
 '/content/model_emb25/model_checkpoint.data-00000-of-00001',
 '/content/model_emb25/model_checkpoint.index']

In [ ]:
model_hate_eval = build_model(base_layers.EVAL)
metrics = create_metrics(LABELS)
model_hate_eval.compile(metrics=metrics)
model_hate_eval.load_weights('model_emb25/model_checkpoint')
result = model_hate_eval.evaluate(x=test_dataset_hate, return_dict=True)

prec = result['precision@0.5/hate']
rec = result['recall@0.5/all']
f1 = 2 * (prec*rec) / (prec+rec)
print(f1)

lsh vectors 1024 

In [ ]:
LABELS = ['hate',]
CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,}
CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)
MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': False,
    'quantize': False, 
    'max_seq_len': 32,
    'max_seq_len_inference': 32,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 1024,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,}
CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
!gdown --id 1htDMrlhCyMLsCsejBoTsoiELuuodoCJb # lsh_dict1
!gdown --id 1xR527W3KfwKRfxBjKWA4Y5HetGmXaB_u # lsh_dict3

In [ ]:
with open('lsh_dict1.pickle', 'rb') as handle:
    lsh_dict1 = pickle.load(handle)
with open('lsh_dict3.pickle', 'rb') as handle:
    lsh_dict3 = pickle.load(handle)

for word in lsh_dict1:
  lsh_dict1[word] = np.concatenate((lsh_dict1[word], lsh_dict3[word]))

In [ ]:
train_hate_emb21 = create_dict_for_tf_emb(clean_train, Y_train, lsh_dict1, emb_len=1024, max_num_words_sen=32)
test_hate_emb21 = create_dict_for_tf_emb(clean_test, Y_test, lsh_dict1, emb_len=1024, max_num_words_sen=32)

train_dataset_hate_emb21 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb21, test_hate_emb21)
test_dataset_hate_emb21 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb21, test_hate_emb21)

105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 32, 1024)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 32, 1024)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb21, test_dataset_hate_emb21, CONFIG, LABELS, 'model_emb21')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 42s 30ms/step - loss: 0.5793 - accuracy: 0.7765
Epoch 2/10
1000/1000 [==============================] - 30s 30ms/step - loss: 0.4639 - accuracy: 0.8859
Epoch 3/10
1000/1000 [==============================] - 30s 30ms/step - loss: 0.4275 - accuracy: 0.9120
Epoch 4/10
1000/1000 [==============================] - 30s 30ms/step - loss: 0.4144 - accuracy: 0.9207
Epoch 5/10
1000/1000 [==============================] - 30s 30ms/step - loss: 0.4082 - accuracy: 0.9251
Epoch 6/10
1000/1000 [==============================] - 30s 30ms/step - loss: 0.4080 - accuracy: 0.9253
Epoch 7/10
1000/1000 [==============================] - 30s 30ms/step - loss: 0.4065 - accuracy: 0.9251
Epoch 8/10
1000/1000 [==============================] - 30s 30ms/step - loss: 0.4078 - accuracy: 0.9233
Epo

In [6]:
gdown.download_folder('https://drive.google.com/drive/folders/1DXKcyrx6vIffgBjF_w_ss0LBy9s3nxSO?usp=sharing', quiet=True)

['/content/model_emb21/checkpoint',
 '/content/model_emb21/model_checkpoint.data-00000-of-00001',
 '/content/model_emb21/model_checkpoint.index']

In [ ]:
model_hate_eval = build_model(base_layers.EVAL)
metrics = create_metrics(LABELS)
model_hate_eval.compile(metrics=metrics)
model_hate_eval.load_weights('test_hate_emb21/model_checkpoint')
result = model_hate_eval.evaluate(x=test_dataset_hate, return_dict=True)

prec = result['precision@0.5/hate']
rec = result['recall@0.5/all']
f1 = 2 * (prec*rec) / (prec+rec)
print(f1)

lsh random vectors 0.51

In [ ]:
LABELS = ['hate',]
CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,}
CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)
MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': False,
    'quantize': False, #
    'max_seq_len': 32,
    'max_seq_len_inference': 32,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 512,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,}
CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
with open('lsh_dict2.pickle', 'rb') as handle:
    lsh_dict2 = pickle.load(handle)

train_hate_emb19 = create_dict_for_tf_emb(clean_train, Y_train, lsh_dict2, emb_len=512, max_num_words_sen=32)
test_hate_emb19 = create_dict_for_tf_emb(clean_test, Y_test, lsh_dict2, emb_len=512, max_num_words_sen=32)

train_dataset_hate_emb19 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb19, test_hate_emb19)
test_dataset_hate_emb19 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb19, test_hate_emb19)

105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 32, 512)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 32, 512)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb19, test_dataset_hate_emb19, CONFIG, LABELS, 'model_emb19')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 33s 23ms/step - loss: 0.5794 - accuracy: 0.7721
Epoch 2/10
1000/1000 [==============================] - 23s 23ms/step - loss: 0.4625 - accuracy: 0.8892
Epoch 3/10
1000/1000 [==============================] - 23s 23ms/step - loss: 0.4283 - accuracy: 0.9109
Epoch 4/10
1000/1000 [==============================] - 23s 23ms/step - loss: 0.4167 - accuracy: 0.9179
Epoch 5/10
1000/1000 [==============================] - 23s 23ms/step - loss: 0.4116 - accuracy: 0.9207
Epoch 6/10
1000/1000 [==============================] - 24s 24ms/step - loss: 0.4106 - accuracy: 0.9213
Epoch 7/10
1000/1000 [==============================] - 24s 24ms/step - loss: 0.4105 - accuracy: 0.9209
Epoch 8/10
1000/1000 [==============================] - 24s 24ms/step - loss: 0.4096 - accuracy: 0.9214
Epo

In [ ]:
model_pred19 = build_model(base_layers.EVAL)
metrics = create_metrics(LABELS)
model_pred19.compile(metrics=metrics)
model_pred19.load_weights(f'model_emb19/model_checkpoint')
result = model_pred19.evaluate(x=test_dataset_hate_emb19, return_dict=True)
prec = result['precision@0.5/hate']
rec = result['recall@0.5/all']
f1 = 2 * (prec*rec) / (prec+rec)
print(f1)

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
9/9 [==============================] - 2s 17ms/step - loss: 0.0000e+00 - precision@0.5/hate: 0.6757 - recall@0.5/hate: 0.4132 - TruePositives@0.5/hate: 50.0000 - TrueNegatives@0.5/hate: 800.0000 - FalsePositives@0.5/hate: 24.0000 - FalseNegatives@0.5/hate: 71.0000 - precision@0.5/all: 0.6757 - recall@0.5/all: 0.4132
0.5128205223070956


rerun with better processing

In [ ]:
LABELS = [
    'hate',
]

CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,
}

CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)

MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': False,
    'quantize': False,
    'max_seq_len': 64,
    'max_seq_len_inference': 64,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 512,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,
}

CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
!gdown --id 1Ve2DVbqL6CYWvrHxc9x1iJznwx8J8WLy # lsh_dict1_rerun

In [ ]:
with open('lsh_dict1_rerun.pickle', 'rb') as handle:
    lsh_dict1 = pickle.load(handle)

In [ ]:
clean_train = clean_dataset_with_emoji(text) #better processing no emoji
clean_test = clean_dataset_with_emoji(text_test)

In [ ]:
train_hate_emb2 = create_dict_for_tf_emb(clean_train, Y_train, lsh_dict1, emb_len=512, max_num_words_sen=64)
test_hate_emb2 = create_dict_for_tf_emb(clean_test, Y_test, lsh_dict1, emb_len=512, max_num_words_sen=64)

train_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb2, test_hate_emb2)
test_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb2, test_hate_emb2)

105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 512)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 512)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb1, test_dataset_hate_emb1, CONFIG, LABELS, 'model_emb_lsh_rerun')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 181s 177ms/step - loss: 0.5833 - accuracy: 0.7686
Epoch 2/10
1000/1000 [==============================] - 162s 162ms/step - loss: 0.4699 - accuracy: 0.8792
Epoch 3/10
1000/1000 [==============================] - 168s 168ms/step - loss: 0.4374 - accuracy: 0.9017
Epoch 4/10
1000/1000 [==============================] - 162s 162ms/step - loss: 0.4264 - accuracy: 0.9071
Epoch 5/10
1000/1000 [==============================] - 163s 163ms/step - loss: 0.4204 - accuracy: 0.9113
Epoch 6/10
1000/1000 [==============================] - 168s 168ms/step - loss: 0.4198 - accuracy: 0.9114
Epoch 7/10
1000/1000 [==============================] - 163s 163ms/step - loss: 0.4196 - accuracy: 0.9116
Epoch 8/10
1000/1000 [==============================] - 163s 163ms/step - loss: 0.4197 - accu

In [ ]:
(83+748)/945

0.8793650793650793

In [ ]:
# model_hate_eval2 = build_model(base_layers.EVAL)
# metrics = create_metrics(LABELS)
# model_hate_eval2.compile(metrics=metrics)
# model_hate_eval2.load_weights('model_emb_lsh_rerun/model_checkpoint')
# result = model_hate_eval2.evaluate(x=test_dataset_hate_emb1, return_dict=True)

# prec = result['precision@0.5/hate']
# rec = result['recall@0.5/all']
# f1 = 2 * (prec*rec) / (prec+rec)
# print(f1)

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
9/9 [==============================] - 2s 57ms/step - loss: 0.0000e+00 - precision@0.5/hate: 0.5220 - recall@0.5/hate: 0.6860 - TruePositives@0.5/hate: 83.0000 - TrueNegatives@0.5/hate: 748.0000 - FalsePositives@0.5/hate: 76.0000 - FalseNegatives@0.5/hate: 38.0000 - precision@0.5/all: 0.5220 - recall@0.5/all: 0.6860
0.5928571455575978


In [ ]:
# save_classification_results(model_hate_eval2, test_dataset_hate_emb1, clean_test, Y_test, 'lsh_best2')

In [7]:
gdown.download_folder('https://drive.google.com/drive/folders/18KU-MbB6a2lBYtqmspibpONK5v1tLksR?usp=sharing', quiet=True)

['/content/model_emb_lsh_rerun/checkpoint',
 '/content/model_emb_lsh_rerun/model_checkpoint.data-00000-of-00001',
 '/content/model_emb_lsh_rerun/model_checkpoint.index']

In [ ]:
model_hate_eval = build_model(base_layers.EVAL)
metrics = create_metrics(LABELS)
model_hate_eval.compile(metrics=metrics)
model_hate_eval.load_weights('model_emb_lsh_rerun/model_checkpoint')
result = model_hate_eval.evaluate(x=test_dataset_hate, return_dict=True)

prec = result['precision@0.5/hate']
rec = result['recall@0.5/all']
f1 = 2 * (prec*rec) / (prec+rec)
print(f1)

### lsh new processing emoji added with train

In [ ]:
with open('lsh_dict1_emoji_rerun.pickle', 'rb') as handle:
    lsh_dict1 = pickle.load(handle)

In [ ]:
!gdown --id 1ajgu8Qg9JyHDSglDAYJUtlSdBLSQW3fi # lsh_dict1_emoji_rerun

In [ ]:
del lsh_dict1

In [ ]:
clean_train = clean_dataset_with_emoji(text, keep_emoji=True) #better processing no emoji
clean_test = clean_dataset_with_emoji(text_test,keep_emoji=True)

train_hate_emb2 = create_dict_for_tf_emb(clean_train, Y_train, lsh_dict1, emb_len=512, max_num_words_sen=64)
test_hate_emb2 = create_dict_for_tf_emb(clean_test, Y_test, lsh_dict1, emb_len=512, max_num_words_sen=64)

train_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb2, test_hate_emb2)
test_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb2, test_hate_emb2)

105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 512)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 512)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb1, test_dataset_hate_emb1, CONFIG, LABELS, 'model_emb_lsh_rerun2')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 34s 20ms/step - loss: 0.5830 - accuracy: 0.7688
Epoch 2/10
1000/1000 [==============================] - 20s 20ms/step - loss: 0.4702 - accuracy: 0.8781
Epoch 3/10
1000/1000 [==============================] - 21s 21ms/step - loss: 0.4361 - accuracy: 0.9007
Epoch 4/10
1000/1000 [==============================] - 20s 20ms/step - loss: 0.4248 - accuracy: 0.9079
Epoch 5/10
1000/1000 [==============================] - 20s 20ms/step - loss: 0.4195 - accuracy: 0.9120
Epoch 6/10
1000/1000 [==============================] - 20s 20ms/step - loss: 0.4188 - accuracy: 0.9118
Epoch 7/10
1000/1000 [==============================] - 20s 20ms/step - loss: 0.4180 - accuracy: 0.9128
Epoch 8/10
1000/1000 [==============================] - 21s 21ms/step - loss: 0.4177 - accuracy: 0.9122
Epo

In [ ]:
(77+767)/945

0.8931216931216931

In [ ]:
# model_hate_eval2 = build_model(base_layers.EVAL)
# metrics = create_metrics(LABELS)
# model_hate_eval2.compile(metrics=metrics)
# model_hate_eval2.load_weights('model_emb_lsh_rerun2/model_checkpoint')
# result = model_hate_eval2.evaluate(x=test_dataset_hate_emb1, return_dict=True)

# prec = result['precision@0.5/hate']
# rec = result['recall@0.5/all']
# f1 = 2 * (prec*rec) / (prec+rec)
# print(f1)

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
9/9 [==============================] - 2s 22ms/step - loss: 0.0000e+00 - precision@0.5/hate: 0.5746 - recall@0.5/hate: 0.6364 - TruePositives@0.5/hate: 77.0000 - TrueNegatives@0.5/hate: 767.0000 - FalsePositives@0.5/hate: 57.0000 - FalseNegatives@0.5/hate: 44.0000 - precision@0.5/all: 0.5746 - recall@0.5/all: 0.6364
0.6039215622732887


In [ ]:
# save_classification_results(model_hate_eval2, test_dataset_hate_emb1, clean_test, Y_test, 'lsh_best')

In [9]:
gdown.download_folder('https://drive.google.com/drive/folders/1ns3nFJeSthN8Kghl--yj9-wjhL-cqpfp?usp=sharing', quiet=True)

['/content/model_emb_lsh_rerun2/checkpoint',
 '/content/model_emb_lsh_rerun2/model_checkpoint.data-00000-of-00001',
 '/content/model_emb_lsh_rerun2/model_checkpoint.index']

In [ ]:
model_hate_eval = build_model(base_layers.EVAL)
metrics = create_metrics(LABELS)
model_hate_eval.compile(metrics=metrics)
model_hate_eval.load_weights('model_emb_lsh_rerun2/model_checkpoint')
result = model_hate_eval.evaluate(x=test_dataset_hate, return_dict=True)

prec = result['precision@0.5/hate']
rec = result['recall@0.5/all']
f1 = 2 * (prec*rec) / (prec+rec)
print(f1)

## glove + PRADO

In [ ]:
LABELS = ['hate',]
CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,}
CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)
MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': False,
    'quantize': False, # maybe True??
    'max_seq_len': 32,
    'max_seq_len_inference': 32,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 128,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,}
CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
glove1 = glove.Glove()
glove1 = glove1.load("glove1.model")

In [ ]:
train_hate_emb22 = create_dict_for_tf_emb(clean_train, Y_train, glove1, emb_len=128, max_num_words_sen=32, glove=True)
test_hate_emb22 = create_dict_for_tf_emb(clean_test, Y_test, glove1, emb_len=128, max_num_words_sen=32, glove=True)

train_dataset_hate_emb22 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb22, test_hate_emb22)
test_dataset_hate_emb22 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb22, test_hate_emb22)

105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 32, 128)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 32, 128)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb22, test_dataset_hate_emb22, CONFIG, LABELS, 'model_emb22')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 31s 20ms/step - loss: 0.6208 - accuracy: 0.6657
Epoch 2/10
1000/1000 [==============================] - 20s 20ms/step - loss: 0.5052 - accuracy: 0.8187
Epoch 3/10
1000/1000 [==============================] - 20s 20ms/step - loss: 0.4741 - accuracy: 0.8639
Epoch 4/10
1000/1000 [==============================] - 20s 20ms/step - loss: 0.4632 - accuracy: 0.8772
Epoch 5/10
1000/1000 [==============================] - 20s 20ms/step - loss: 0.4597 - accuracy: 0.8809
Epoch 6/10
1000/1000 [==============================] - 20s 20ms/step - loss: 0.4583 - accuracy: 0.8812
Epoch 7/10
1000/1000 [==============================] - 20s 20ms/step - loss: 0.4580 - accuracy: 0.8828
Epoch 8/10
1000/1000 [==============================] - 20s 20ms/step - loss: 0.4577 - accuracy: 0.8830
Epo

In [ ]:
LABELS = [
    'hate',
]
CONFIG = {
    'name': 'models.prado',
    'batch_size': 105,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,
}
CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)
MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': False,
    'quantize': False, 
    'max_seq_len': 64,
    'max_seq_len_inference': 64,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 128,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,

}
CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
del lsh_dict1

In [ ]:
glove1 = glove.Glove()
glove1 = glove1.load("glove2.model")

In [ ]:
clean_train = clean_dataset_with_emoji(text) 
clean_test = clean_dataset_with_emoji(text_test)

train_hate_emb2 = create_dict_for_tf_emb(clean_train, Y_train, glove1, emb_len=128, max_num_words_sen=64, glove=True)
test_hate_emb2 = create_dict_for_tf_emb(clean_test, Y_test, glove1, emb_len=128, max_num_words_sen=64, glove=True)

train_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.TRAIN, train_hate_emb2, test_hate_emb2)
test_dataset_hate_emb1 = build_dataset_poleval_emb(base_layers.EVAL, train_hate_emb2, test_hate_emb2)

105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 128)
(105,)
105
type of features<class 'dict'> type of text <class 'tensorflow.python.framework.ops.Tensor'>
(105, 64, 128)
(105,)


In [ ]:
# build_run_save_eval_model(train_dataset_hate_emb1, test_dataset_hate_emb1, CONFIG, LABELS, 'model_emb_glove_rerun')

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
Epoch 1/10
1000/1000 [==============================] - 131s 129ms/step - loss: 0.6176 - accuracy: 0.6692
Epoch 2/10
1000/1000 [==============================] - 131s 131ms/step - loss: 0.5031 - accuracy: 0.8211
Epoch 3/10
1000/1000 [==============================] - 128s 128ms/step - loss: 0.4720 - accuracy: 0.8635
Epoch 4/10
1000/1000 [==============================] - 145s 145ms/step - loss: 0.4620 - accuracy: 0.8759
Epoch 5/10
1000/1000 [==============================] - 127s 127ms/step - loss: 0.4583 - accuracy: 0.8801
Epoch 6/10
1000/1000 [==============================] - 127s 127ms/step - loss: 0.4570 - accuracy: 0.8817
Epoch 7/10
1000/1000 [==============================] - 130s 130ms/step - loss: 0.4566 - accuracy: 0.8803
Epoch 8/10
1000/1000 [==============================] - 131s 131ms/step - loss: 0.4565 - accu

9/9 [==============================] - 2s 40ms/step - loss: 0.0000e+00 - precision@0.5/hate: 0.4180 - recall@0.5/hate: 0.4215 - TruePositives@0.5/hate: 51.0000 - TrueNegatives@0.5/hate: 753.0000 - FalsePositives@0.5/hate: 71.0000 - FalseNegatives@0.5/hate: 70.0000 - precision@0.5/all: 0.4180 - recall@0.5/all: 0.4215
0.4197530887747063


In [ ]:
(51+753)/945

0.8507936507936508

In [ ]:
# model_hate_eval2 = build_model(base_layers.EVAL)
# metrics = create_metrics(LABELS)
# model_hate_eval2.compile(metrics=metrics)
# model_hate_eval2.load_weights('model_emb_glove_rerun/model_checkpoint')
# result = model_hate_eval2.evaluate(x=test_dataset_hate_emb1, return_dict=True)

# prec = result['precision@0.5/hate']
# rec = result['recall@0.5/all']
# f1 = 2 * (prec*rec) / (prec+rec)
# print(f1)

KerasTensor(type_spec=TensorSpec(shape=(None,), dtype=tf.float32, name='sequence_length'), name='sequence_length', description="created by layer 'sequence_length'")
9/9 [==============================] - 1s 15ms/step - loss: 0.0000e+00 - precision@0.5/hate: 0.4180 - recall@0.5/hate: 0.4215 - TruePositives@0.5/hate: 51.0000 - TrueNegatives@0.5/hate: 753.0000 - FalsePositives@0.5/hate: 71.0000 - FalseNegatives@0.5/hate: 70.0000 - precision@0.5/all: 0.4180 - recall@0.5/all: 0.4215
0.4197530887747063


In [ ]:
save_classification_results(model_hate_eval2, test_dataset_hate_emb1, clean_test, Y_test, 'glove_best')

In [ ]:
gdown.download_folder('https://drive.google.com/drive/folders/1-BoGnwW8NTkgID-03LQ9YKNGOJXIVUs1?usp=sharing', quiet=True)

['/content/model_emb_lsh_rerun2/checkpoint',
 '/content/model_emb_lsh_rerun2/model_checkpoint.data-00000-of-00001',
 '/content/model_emb_lsh_rerun2/model_checkpoint.index']

In [ ]:
model_hate_eval = build_model(base_layers.EVAL)
metrics = create_metrics(LABELS)
model_hate_eval.compile(metrics=metrics)
model_hate_eval.load_weights('model_emb_glove_rerun/model_checkpoint')
result = model_hate_eval.evaluate(x=test_dataset_hate, return_dict=True)

prec = result['precision@0.5/hate']
rec = result['recall@0.5/all']
f1 = 2 * (prec*rec) / (prec+rec)
print(f1)

## comparison section

In [ ]:
clean_train = clean_dataset_with_emoji(text, keep_emoji=True) #better processing no emoji
clean_test = clean_dataset_with_emoji(text_test,keep_emoji=True)

In [ ]:
with open('word2vec_best.pickle', 'rb') as handle:
    word2vec_best = pickle.load(handle)
with open('glove_best.pickle', 'rb') as handle:
    glove_best = pickle.load(handle)
with open('lsh_best.pickle', 'rb') as handle:
    lsh_best = pickle.load(handle)
with open('lsh_best2.pickle', 'rb') as handle:
    lsh_best2 = pickle.load(handle)
with open('fasttext_best.pickle', 'rb') as handle:
    fasttext_best = pickle.load(handle)
with open('prado_best.pickle', 'rb') as handle:
    prado_best = pickle.load(handle)

In [ ]:
for item in word2vec_best['tp']:
  if item not in fasttext_best['tp'] and item not in glove_best['tp'] and item not in lsh_best['tp'] and item not in prado_best['tp']:
    print(item) 

chcieć aby tak być miernota
w powietrze sa środki chemiczny uważać wygladasz na podatny
rozjebać kc raz z i on ojciec tylko że wy to się tylko śnić
gdzie być to cały zakłamanelewackie skurwysynstwo no gdzie
piskomuna zdychać w lękachnnasze słowo on udręka
ty załgany łachudrojakby nie brejza to nikt by o tym nie wiedzieć
jak widzieć były premiera marcinkiewicz to zawsze myśleć qrwajak kto taki móc bycie premier taki skończony błazen
to pieprzyć komunał pad że być wartobudzi w ja niesmak drugi tusk qrwa
a może co o pmm który sprowokować zachować sięjak typowy pisowski nieuk i idiota
i tym sposób zwyciężczyni turniej o plastusiowego debil zostać agnieszka holland brawo


In [ ]:
for item in fasttext_best['tp']:
  if item not in word2vec_best['tp'] and item not in glove_best['tp'] and item not in lsh_best['tp'] and item not in prado_best['tp']:
    print(item)

jaki tama profesor to kangur który aborygen zamienić w człowiek
trzymać bo on postraszyć że jak nie to być on tuczyć jak gęś


In [ ]:
for item in glove_best['tp']:
  if item not in word2vec_best['tp'] and item not in fasttext_best['tp'] and item not in lsh_best['tp'] and item not in prado_best['tp']:
    print(item)

adrian juda figurant wieś i lobby żydowski
jak widzieć były premiera marcinkiewicz to zawsze myśleć qrwa jak kto taki móc bycie premier taki skończony błazen
a kto traktować poważnie ten szmatławiec


In [ ]:
for item in lsh_best['tp']:
  if item not in word2vec_best['tp'] and item not in fasttext_best['tp'] and item not in glove_best['tp'] and item not in prado_best['tp']:
    print(item)

z to co wiedzieć to wyłowić on ciało z jakiś woda
zamknąć mu jeszcze dostęp do wc niech sikać do kwiatek


In [ ]:
for item in prado_best['tp']:
  if item not in word2vec_best['tp'] and item not in fasttext_best['tp'] and item not in glove_best['tp'] and item not in lsh_best['tp']:
    print(item)

ten szmat się nie komentować
podstawowy zadać każdy ksiądz to narzucać wiara
z trolami nie dyskutować a z głupi zwłaszcza
hehe do gaz hehe marzec 2018
jezdzisz na wózek tramwaj uciał ci noga
kwalifikacje pogodynka umożliwiać w pis wszystek


In [ ]:
for item in word2vec_best['tp']:
  if item not in prado_best['tp']:
    print(item)

widać człowiek w sącz tak pasować ja by mu w gęba napluć
i wojna gadowski z psychiatra
chcieć aby tak być miernota
w powietrze sa środki chemiczny uważać wygladasz na podatny
rozjebać kc raz z i on ojciec tylko że wy to się tylko śnić
złodziejstwo kurestwo patologia kompleks brak pewność siebie
gdzie być to cały zakłamanelewackie skurwysynstwo no gdzie
ja dalej dziwić ale ciągły wasze ujadać skamleć donosić i kablować na polska kretyn
twój chłop się jeszcze nie powiesić ja by już dyndać jak by miał taki idiotka
ola wiedzieć że chodzić do burdel nadawać pan w kategoria zjebać rok 2018 pierwszy miejsce
jaka obywatel kategorycznie domagać się aby włożyć soba nos w końcowy odcinek układ pokarmowy
piskomuna zdychać w lękachnnasze słowo on udręka
być tym z wodogłowie nie mówić nikt
powinien wrócić do szambo z który wyleźć
ty załgany łachudrojakby nie brejza to nikt by o tym nie wiedzieć
o czy być łgać jutro
jak widzieć były premiera marcinkiewicz to zawsze myśleć qrwajak kto taki móc bycie p

In [ ]:
for item in prado_best['tp']:
  if item not in word2vec_best['tp']:
    print(item)

ten szmat się nie komentować
dokładnie pisdzielstwo nie mój prawa rozpierdalać system sądownictwo nie maić większość
półgłówek wieliński wymyślić soba półautorytaryzm
podstawowy zadać każdy ksiądz to narzucać wiara
rozumieć że standardowy polski rodzina to pierdolnąć stary kawaler zoofil i kot
gdyby być u władza spałować taki marsz
on być na poziom pierdolnąć zaawansowany więc nie dziwić nic
z trolami nie dyskutować a z głupi zwłaszcza
ty załgany łachudra jakby nie brejza to nikt by o tym nie wiedzieć
i tak dobrnąć na dno szambo
hehe do gaz hehe marzec 2018
a może co o pmm który sprowokować zachować się jak typowy pisowski nieuk i idiota
wysoki porać zaistnieć w 21 wiek a wy odizolować od zdrowy częsty społeczeństwo o nie pozwolić na chory indoktrynacja
jezdzisz na wózek tramwaj uciał ci noga
i tym sposób zwyciężczyni turniej o plastusiowego debil zostać agnieszka holland brawo uśmiech
ale ty być jebnięty rzadko spotykać egzemplarz nie uwarzasz że im to wisieć
podobno przed sejm ustawi

In [ ]:
for item in fasttext_best['tp']:
  if item not in word2vec_best['tp']:
    print(item)

dokładnie pisdzielstwo nie mój prawa rozpierdalać system sądownictwo nie maić większość
rozumieć że standardowy polski rodzina to pierdolnąć stary kawaler zoofil i kot
gdzie być to cały zakłamany lewacki skurwysynstwo no gdzie
jaki tama profesor to kangur który aborygen zamienić w człowiek
on być na poziom pierdolnąć zaawansowany więc nie dziwić nic
ten pierwszy pójść na wojna być my bronić patriota nawet miska ryż nie chcieć
piskomuna zdychać w lęk nnasze słowo on udręka
ty załgany łachudra jakby nie brejza to nikt by o tym nie wiedzieć
i tak dobrnąć na dno szambo
to pieprzyć komunał pad że być warta budzić w ja niesmak drugi tusk qrwa
a może co o pmm który sprowokować zachować się jak typowy pisowski nieuk i idiota
i tym sposób zwyciężczyni turniej o plastusiowego debil zostać agnieszka holland brawo uśmiech
trzymać bo on postraszyć że jak nie to być on tuczyć jak gęś
podobno przed sejm ustawić gilotyna maja obcinać ręka ubywatelom rp ręka który podnieść na władza
bankster od frankow

In [ ]:
for item in word2vec_best['tp']:
  if item not in fasttext_best['tp']:
    print(item)

chcieć aby tak być miernota
w powietrze sa środki chemiczny uważać wygladasz na podatny
rozjebać kc raz z i on ojciec tylko że wy to się tylko śnić
konserwator kuweta gegacz i przechył mózgowy za pieniądz robić lód w tokfm
gdzie być to cały zakłamanelewackie skurwysynstwo no gdzie
ty szambo ktore mysli z być wallenrod spuszczać jeden ruch i polecać
piskomuna zdychać w lękachnnasze słowo on udręka
powinien wrócić do szambo z który wyleźć
ty załgany łachudrojakby nie brejza to nikt by o tym nie wiedzieć
o czy być łgać jutro
jak widzieć były premiera marcinkiewicz to zawsze myśleć qrwajak kto taki móc bycie premier taki skończony błazen
to pieprzyć komunał pad że być wartobudzi w ja niesmak drugi tusk qrwa
a może co o pmm który sprowokować zachować sięjak typowy pisowski nieuk i idiota
i tym sposób zwyciężczyni turniej o plastusiowego debil zostać agnieszka holland brawo
znowu łgać i tak ciągło bez ustanek
ty być jebnięty tło taki parch mamić jechać przez pół polski


In [ ]:
for i in range(len( clean_test)):
  if int(Y_test[i])==1:
    item = clean_test[i]
    if item not in word2vec_best['tp'] and item not in fasttext_best['tp'] and item not in glove_best['tp'] and item not in lsh_best['tp'] and item not in prado_best['tp']:
      print(item)

powinien odpowiedzieć za działanie na szkoda polski i obywatel
na szczęście i on partia zniknąć wkrótce nze scena polityczny brawo
tym który nie być na mecz z wzgląd na koniec zgoda mamić kilka słowić do przekazać wypierdalać do wrocław alba gdańska
szkoda z on być bezmózgowiec
być 1 zjebem 4 rp
a potem pójść pod prokurator za ukraść kilometrówka
jak sprawić zgb dalej nie mieć wjazd jak proces królowa zalaci
nie kłócić się jak nie chcieć miecić późno problem
przecież to prosta wystarczyć nie kraść
właśnie nie xd być wstrząsnąć głupota ten osoba i wazystkich inny który ja popierać
zrzec się polskie obywatelstwo dla karier pogratulować
niestety nie oglądać ale jednać rzecz w tym post być nieprawdziwy żalek i wiedza to dwa wykluczać się byt
ssać wszystek otwór co tylko dać się wessać
nie żreć tył to nie być mieć obstrukcja
a petru to chyba tylko po to żeby widz wkurwić
a dlaczego bać się kretynka nauczyciel
i bardzo dobrze sprzedać polska za pięć stówka pis to mat właśnie to na co zasługi

In [ ]:
for item in word2vec_best['tp']:
  if item not in prado_best['tp']:
    print(item)

widać człowiek w sącz tak pasować ja by mu w gęba napluć
i wojna gadowski z psychiatra
chcieć aby tak być miernota
w powietrze sa środki chemiczny uważać wygladasz na podatny
rozjebać kc raz z i on ojciec tylko że wy to się tylko śnić
złodziejstwo kurestwo patologia kompleks brak pewność siebie
gdzie być to cały zakłamanelewackie skurwysynstwo no gdzie
ja dalej dziwić ale ciągły wasze ujadać skamleć donosić i kablować na polska kretyn
twój chłop się jeszcze nie powiesić ja by już dyndać jak by miał taki idiotka
ola wiedzieć że chodzić do burdel nadawać pan w kategoria zjebać rok 2018 pierwszy miejsce
jaka obywatel kategorycznie domagać się aby włożyć soba nos w końcowy odcinek układ pokarmowy
piskomuna zdychać w lękachnnasze słowo on udręka
być tym z wodogłowie nie mówić nikt
powinien wrócić do szambo z który wyleźć
ty załgany łachudrojakby nie brejza to nikt by o tym nie wiedzieć
o czy być łgać jutro
jak widzieć były premiera marcinkiewicz to zawsze myśleć qrwajak kto taki móc bycie p

In [ ]:
dataset1 = prado_best
dataset2 = word2vec_best
for cat in ['tp', 'fp', 'fn']:
  k=0
  print(len(dataset1[cat]), len(dataset2[cat]))

  print(f'CHECKING CAT {cat} DIFFERENCES')
  for item in dataset1[cat]:
    if item not in dataset2[cat]:
      print(item)
      k+=1
  print(k)
  print()

  k=0
  for item in dataset2[cat]:
    if item not in dataset1[cat]:
      print(item)
      k+=1
  print(k)
  print()

67 73
CHECKING CAT tp DIFFERENCES
ten szmat się nie komentować
dokładnie pisdzielstwo nie mój prawa rozpierdalać system sądownictwo nie maić większość
półgłówek wieliński wymyślić soba półautorytaryzm
podstawowy zadać każdy ksiądz to narzucać wiara
rozumieć że standardowy polski rodzina to pierdolnąć stary kawaler zoofil i kot
gdyby być u władza spałować taki marsz
on być na poziom pierdolnąć zaawansowany więc nie dziwić nic
z trolami nie dyskutować a z głupi zwłaszcza
ty załgany łachudra jakby nie brejza to nikt by o tym nie wiedzieć
i tak dobrnąć na dno szambo
hehe do gaz hehe marzec 2018
a może co o pmm który sprowokować zachować się jak typowy pisowski nieuk i idiota
wysoki porać zaistnieć w 21 wiek a wy odizolować od zdrowy częsty społeczeństwo o nie pozwolić na chory indoktrynacja
jezdzisz na wózek tramwaj uciał ci noga
i tym sposób zwyciężczyni turniej o plastusiowego debil zostać agnieszka holland brawo uśmiech
ale ty być jebnięty rzadko spotykać egzemplarz nie uwarzasz że im t

In [ ]:
dataset1 = fasttext_best
dataset2 = lsh_best
for cat in ['tp', 'fp', 'fn']:
  k=0
  print(len(dataset1[cat]), len(dataset2[cat]))

  print(f'CHECKING CAT {cat} DIFFERENCES')
  for item in dataset1[cat]:
    if item not in dataset2[cat]:
      print(item)
      k+=1
  print(k)
  print()

  k=0
  for item in dataset2[cat]:
    if item not in dataset1[cat]:
      print(item)
      k+=1
  print(k)
  print()

72 77
CHECKING CAT tp DIFFERENCES
jaki tama profesor to kangur który aborygen zamienić w człowiek
jaka obywatel kategorycznie domagać się aby włożyć soba nos w końcowy odcinek układ pokarmowy
być tym z wodogłowie nie mówić nikt
trzymać bo on postraszyć że jak nie to być on tuczyć jak gęś
podobno przed sejm ustawić gilotyna maja obcinać ręka ubywatelom rp ręka który podnieść na władza
czemu zapraszać rasista i zero bezwzględny
ten mała pizda być na bateria napędzać pizda
ukraść warszawiak podstępnie plac piłsudski samowolnie i bezprawnie postawić pomnik taki to wasze starać
8

z to co wiedzieć to wyłowić on ciało z jakiś woda
konserwator kuweta gegacz i przechył mózgowy za pieniądz robić lód w tokfm
gdyby być u władza spałować taki marsz
ty szambo ktore mysli z być wallenrod spuszczać jeden ruch i polecać
powinien wrócić do szambo z który wyleźć
naraz to poprosić kolega niech ci pociągnąć delegalizacja
o czy być łgać jutro
ale ty być jebnięty rzadko spotykać egzemplarz nie uwarzasz że i

In [ ]:
lsh_best['fn']

['ten szmat się nie komentować',
 'adrian juda figurant wieś i lobby żydowski',
 'powinien odpowiedzieć za działanie na szkoda polski i obywatel',
 'półgłówek wieliński wymyślić soba półautorytaryzm',
 'podstawowy zadać każdy ksiądz to narzucać wiara',
 'na szczęście i on partia zniknąć wkrótce nze scena polityczny brawo',
 'chcieć aby tak być miernota',
 'w powietrze sa środki chemiczny uważać wygladasz na podatny',
 'tym który nie być na mecz z wzgląd na koniec zgoda mamić kilka słowić do przekazać wypierdalać do wrocław alba gdańska',
 'szkoda z on być bezmózgowiec',
 'rozjebać kc raz z i on ojciec tylko że wy to się tylko śnić',
 'być 1 zjebem 4 rp',
 'a potem pójść pod prokurator za ukraść kilometrówka',
 'jak sprawić zgb dalej nie mieć wjazd jak proces królowa zalaci',
 'nie kłócić się jak nie chcieć miecić późno problem',
 'przecież to prosta wystarczyć nie kraść',
 'właśnie nie xd być wstrząsnąć głupota ten osoba i wazystkich inny który ja popierać',
 'zrzec się polskie obywate